### Project Chicago: Transform ResStock PACKAGE Results to Multi-family (RECS) Building Types
Created on: 01/07/2020 \
By: Lixi Liu (Lixi.Liu@nrel.gov)

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
print(f'Notebook path: {os.getcwd()}')

Notebook path: /Users/lliu2/Documents/GitHub/ResStock/files


### Download results online
* unprocessed upgrade results: S3/resbldg-datasets/chicagoeui
* processed result tables for plots: https://nrel.sharepoint.com/sites/ChicagoRetrofits/Shared%20Documents/Forms/AllItems.aspx?viewid=289cdd1a%2D97c9%2D4bcc%2D8416%2Dc19bf01c6302&id=%2Fsites%2FChicagoRetrofits%2FShared%20Documents%2FGeneral%2FUpgrade%20results

### Initialize
For modifying plots, can go directly to section 2.3 after initialization

In [2]:
# local path to downloaded results
iteration = '02_mf' # <----- options: '01'
iter_path = f'cookcnty_packages_{iteration}'
result_dir = '/Users/lliu2/Documents/Chicago retrofits/ResStock results'
result_path = os.path.join(result_dir, iter_path)

# accessory file path (for adding to package iteration)
acc_iter_path = f'cookcnty_upgrades_02'
acc_path = os.path.join(result_dir, acc_iter_path)

## create folder for post-processed results:
if not os.path.exists(os.path.join(result_path, 'processed results')):
    os.mkdir(os.path.join(result_path, 'processed results'))
    
## create folder for exported baseline and upgrade results to csv (to share with Elevate):
combined_res_csv_path = os.path.join(result_path, 'processed results', 'raw combined csvs')
if not os.path.exists(combined_res_csv_path):
    os.mkdir(combined_res_csv_path)
    
## create folder for plots:
plot_path = os.path.join(result_path, 'processed results', 'plots')
if not os.path.exists(plot_path):
    os.mkdir(plot_path)

print(f'Results path: \n   {result_path}')
print(f'Accessory files path: \n   {acc_path}')


Results path: 
   /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_mf
Accessory files path: 
   /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_upgrades_02


### Functions

In [3]:
def get_per_unit_sim_output(df, ref):
    """
    ref (df): baseline df
    """
    cols = [x for x in df.columns if 
            x.endswith('_kwh') or
            x.endswith('_mbtu') or
            x.endswith('_therm') or
            x.endswith('_cost_usd') or
            x.endswith('_ft_2')
           ]
    res = ref.set_index('building_id').reindex(df['building_id']).reset_index()
    df.loc[:, cols] = df.loc[:, cols].replace([None,''],np.nan).divide(
        res['build_existing_model.units_represented'], axis=0)
    
    return df
print('func loaded: "get_per_unit_sim_output"')

def get_per_unit_sim_output_limited(df, ref):
    """
    To reduce computing time
    ref (df): baseline df
    """
    cols = ['simulation_output_report.total_site_natural_gas_therm',
            'simulation_output_report.total_site_electricity_kwh',
            'simulation_output_report.total_site_energy_mbtu',
            'simulation_output_report.upgrade_cost_usd',
            'simulation_output_report.floor_area_conditioned_ft_2'
           ]
    res = ref.set_index('building_id').reindex(df['building_id']).reset_index()
    df.loc[:, cols] = df.loc[:, cols].replace([None,''],np.nan).divide(
        res['build_existing_model.units_represented'], axis=0)
    
    return df
print('func loaded: "get_per_unit_sim_output_limited"')

def add_sqft_eui(df, ref):
    """
    ARG:
        ref (df): baseline df
    RETURN:
        df with added cols: 'sqft', 'gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_kbtupersqft'
    """
    res = ref.set_index('building_id').reindex(df['building_id']).reset_index()
    df['sqft'] = res['simulation_output_report.floor_area_conditioned_ft_2']
    df['gas_eui_thermpersqft'] = df['simulation_output_report.total_site_natural_gas_therm'].divide(df['sqft']) # therm/sqft
    df['elec_eui_kwhpersqft'] = df['simulation_output_report.total_site_electricity_kwh'].divide(df['sqft']) # kwh/sqft
    df['site_eui_kbtupersqft'] = df['simulation_output_report.total_site_energy_mbtu'].divide(df['sqft'])*1000 # kbtu/sqft
    
    for col in ['sqft','gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_kbtupersqft']:
        df.loc[df['simulation_output_report.applicable']==False, col] = np.nan
    
    return df
print('func loaded: "add_sqft_eui"')

def get_res_by_prototype(df, filter_by_df, row):
    """
    ARG:
        df (dataframe): df to slice on
        filter_by_df (dataframe): df used to do the slicing
    RETURN:
        filtered df (dataframe)
    """
    slice_by_df = filter_by_df.copy().set_index('building_id').reindex(df['building_id']).reset_index()
    res_group_i = df[slice_by_df['build_existing_model.geometry_stories'].isin(row['Stories'].split(',')) & \
        slice_by_df['build_existing_model.geometry_wall_type'].isin(row['WallType'].split(',')) & \
        slice_by_df['build_existing_model.vintage_acs'].isin(row['Vintage'].split(',')) & \
        slice_by_df['build_existing_model.geometry_building_type_recs'].isin(['Single-Family Detached'])]
    res_group_i = res_group_i[res_group_i['completed_status']=="Success"]
    
    return res_group_i
print('func loaded: "get_res_by_prototype"')

def load_upgrade(n, file_dir=result_path):
    """
    ARG:
        n (int, str): upgrade number
        file_dir (str): folder in which upgrade can be found, default to main result dir
    RETURN:
        df (dataframe) of upgrade n
    """
    df = pd.read_parquet(os.path.join(file_dir,'upgrades',
                                     f'upgrade={n}/results_up{n:02d}.parquet'))
    return df
print('func loaded: "load_upgrade"')



func loaded: "get_per_unit_sim_output"
func loaded: "get_per_unit_sim_output_limited"
func loaded: "add_sqft_eui"
func loaded: "get_res_by_prototype"
func loaded: "load_upgrade"


### 1. BASELINE results

In [4]:
save_a_copy_in_csv = False # <-----
res = pd.read_parquet(os.path.join(result_path,'baseline','results_up00.parquet'))

# (1) get sqft, gas/elec/site eui
res = add_sqft_eui(res, res)
res['build_existing_model.sample_weight'] = 2173432/40000

if save_a_copy_in_csv:
    res.to_csv(os.path.join(combined_res_csv_path,'results_baseline.csv'), index=False)
    
# (1) get sim output at the unit level (req for MF)
res = get_per_unit_sim_output(res, res)

# (2) get sqft, gas/elec/site eui - redo to get unit-level results
res = add_sqft_eui(res, res)

# check
Nbldgs = 20759
jobs_missing = set(range(1,246)) - set(res.job_id.unique())
print(f'- {len(jobs_missing)} jobs missing: {jobs_missing}')
print(f'- {Nbldgs-len(res)} buildings ({((Nbldgs-len(res))/Nbldgs*100):.2f}%)')

res

- 0 jobs missing: set()
- 0 buildings (0.00%)


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,sqft,gas_eui_thermpersqft,elec_eui_kwhpersqft,site_eui_kbtupersqft
0,1,188,2021-02-14 08:59:53,2021-02-14 09:01:21,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,15.636364,17.909091,35.439280,27.020355,4793.0,39.621348,1138.0,1.361800,9.549458,168.731619
1,2,123,2021-02-14 15:22:23,2021-02-14 15:37:42,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,16.272727,17.363636,157.113193,111.155287,5128.0,191.858633,1138.0,0.452631,4.967620,62.202515
2,3,175,2021-02-15 01:22:58,2021-02-15 01:26:24,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,16.181818,18.181818,102.139434,84.674223,6471.0,108.604937,2115.0,0.589579,5.329544,77.129027
3,4,119,2021-02-14 09:19:39,2021-02-14 09:21:10,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,15.454545,17.909091,22.218382,13.839910,4932.0,24.872680,853.0,0.771225,7.795249,103.702632
4,5,63,2021-02-14 10:36:00,2021-02-14 10:55:40,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,14.272727,10.818182,259.567395,686.250477,619.0,879.114703,853.0,0.000000,16.980124,57.938542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,35,2021-02-14 08:02:24,2021-02-14 08:09:40,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,15.090909,10.454545,215.333442,308.773118,873.0,407.545486,1138.0,0.018703,21.472143,75.135820
20755,20756,219,2021-02-14 12:10:30,2021-02-14 12:54:43,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,12.909091,13.636364,669.908277,988.562169,641.0,1094.666577,617.0,0.031215,20.475466,72.985912
20756,20757,223,2021-02-14 14:28:38,2021-02-14 14:43:34,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,14.545455,13.636364,757.604427,672.887124,5103.0,886.634439,1138.0,0.424643,5.293655,60.516822
20757,20758,243,2021-02-15 00:30:42,2021-02-15 00:41:24,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,14.636364,6.000000,167.760128,778.660057,630.0,893.685975,853.0,0.000000,19.615695,66.931478


In [5]:
# assign heating/cooling
cooling_col = 'build_existing_model.hvac_cooling_type'
heating_col = 'build_existing_model.hvac_heating_type_and_fuel'
print('selected')

# if using a national run
if iteration == '_national_2018':
    res = res[res['build_existing_model.ahs_region']=='CBSA Chicago-Naperville-Elgin, IL-IN-WI'].reset_index(drop=True)
    res
    

selected


### 1.1. Check housing charateristics distributions in BASELINE

In [6]:
proto_chars = ['build_existing_model.geometry_stories',
              'build_existing_model.geometry_wall_type',
              'build_existing_model.vintage_acs',
              cooling_col,
              heating_col,
              'build_existing_model.geometry_floor_area'
             ]

Ns = len(res.query('completed_status=="Success"')); N = len(res)
print(f'>>> ResStock - {iter_path} - BASELINE result summary:\n')
print(f'  * {Ns} / {N} samples ran successfully, {N-Ns} failed, efficacy: {Ns/N:.1%} \n')

print('>>> Housing characteristics splits:\n')

Res_char = []
for i, char in enumerate(proto_chars,1):
    Nchar = res.groupby(char)['building_id'].count()
    Nchar = Nchar/Ns
    Nchar['N_failed'] = len(res[res[char].isnull()])
    print(f'  * [{i}] {Nchar}\n')
    
    ## append for export
    Nchar = Nchar.rename('fraction').to_frame()
    Nchar['housing_char'] = Nchar.index.name
    Res_char.append(Nchar)
    
Res_char = pd.concat(Res_char, axis=0)
Res_char.index.name = 'sub_char'
Res_char = Res_char.reset_index()
Res_char = Res_char[['housing_char','sub_char','fraction']]

Res_char.to_csv(os.path.join(result_path, 'baseline', 'baseline_housing_char_breakdown.csv'), index=False)
print(f'>>> file saved to:\n  {os.path.join(result_path, "baseline")}')

>>> ResStock - cookcnty_packages_02_mf - BASELINE result summary:

  * 20759 / 20759 samples ran successfully, 0 failed, efficacy: 100.0% 

>>> Housing characteristics splits:

  * [1] build_existing_model.geometry_stories
1           0.414374
2           0.200491
3           0.385134
N_failed    0.000000
Name: building_id, dtype: float64

  * [2] build_existing_model.geometry_wall_type
Masonry     0.877547
WoodStud    0.122453
N_failed    0.000000
Name: building_id, dtype: float64

  * [3] build_existing_model.vintage_acs
1940-59     0.147888
1960-79     0.248326
1980-99     0.141963
2000-09     0.091141
2010s       0.015560
<1940       0.355123
N_failed    0.000000
Name: building_id, dtype: float64

  * [4] build_existing_model.hvac_cooling_type
Central AC    0.422756
Heat Pump     0.041476
None          0.088588
Room AC       0.447180
N_failed      0.000000
Name: building_id, dtype: float64

  * [5] build_existing_model.hvac_heating_type_and_fuel
Electricity ASHP                    

### 1.2. Add additional metrics to Baseline

In [7]:
## EE prototype tags
res['vintage_ee'] = '3: post-1978'
res.loc[res['build_existing_model.vintage_acs'].isin(['1940-59','1960-79']),'vintage_ee'] = '2: 1942-1978'
res.loc[res['build_existing_model.vintage_acs']=='<1940','vintage_ee'] = '1: pre-1942'

res['stories'] = '2: 2+ stories'
res.loc[res['build_existing_model.geometry_stories']=='1','stories'] = '1: <2 stories'

print('new cols added to "res"')
res

new cols added to "res"


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,sqft,gas_eui_thermpersqft,elec_eui_kwhpersqft,site_eui_kbtupersqft,vintage_ee,stories
0,1,188,2021-02-14 08:59:53,2021-02-14 09:01:21,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,35.439280,27.020355,4793.0,39.621348,1138.0,1.361800,9.549458,168.731619,2: 1942-1978,1: <2 stories
1,2,123,2021-02-14 15:22:23,2021-02-14 15:37:42,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,157.113193,111.155287,5128.0,191.858633,1138.0,0.452631,4.967620,62.202515,2: 1942-1978,2: 2+ stories
2,3,175,2021-02-15 01:22:58,2021-02-15 01:26:24,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,102.139434,84.674223,6471.0,108.604937,2115.0,0.589579,5.329544,77.129027,1: pre-1942,1: <2 stories
3,4,119,2021-02-14 09:19:39,2021-02-14 09:21:10,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,22.218382,13.839910,4932.0,24.872680,853.0,0.771225,7.795249,103.702632,2: 1942-1978,1: <2 stories
4,5,63,2021-02-14 10:36:00,2021-02-14 10:55:40,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,259.567395,686.250477,619.0,879.114703,853.0,0.000000,16.980124,57.938542,3: post-1978,2: 2+ stories
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,35,2021-02-14 08:02:24,2021-02-14 08:09:40,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,215.333442,308.773118,873.0,407.545486,1138.0,0.018703,21.472143,75.135820,2: 1942-1978,2: 2+ stories
20755,20756,219,2021-02-14 12:10:30,2021-02-14 12:54:43,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,669.908277,988.562169,641.0,1094.666577,617.0,0.031215,20.475466,72.985912,3: post-1978,2: 2+ stories
20756,20757,223,2021-02-14 14:28:38,2021-02-14 14:43:34,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,757.604427,672.887124,5103.0,886.634439,1138.0,0.424643,5.293655,60.516822,3: post-1978,2: 2+ stories
20757,20758,243,2021-02-15 00:30:42,2021-02-15 00:41:24,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,167.760128,778.660057,630.0,893.685975,853.0,0.000000,19.615695,66.931478,3: post-1978,2: 2+ stories


## 2A. PACKAGE post-processing ###
Three packages were ran as partitioned files. See "Cookcnty_packages_02.yml" for info
#### (1) Packages WITHOUT partitions (N=1, pkg 06)

In [8]:
def check_and_correct_for_missing_buildings(Pi, res, upgrade_no):
    delta = len(res)-len(Pi)
    if delta>0:
        print(f'   - upgrade_{upgrade_no:02d} is missing {delta} buildings compared to baseline')
    # always reindex
    Pi = Pi.set_index('building_id').reindex(res['building_id']).reset_index()
        
    return Pi

'func "check_and_correct_for_missing_buildings" loaded'

'func "check_and_correct_for_missing_buildings" loaded'

In [9]:
### (1) 
upgrade_list = [1,]
package_numbers = [6,]

print(f'packages without partitions: {len(upgrade_list)}')

for up, pk in zip(upgrade_list, package_numbers):
    file = f'package{pk:02d}.csv'
    filename = os.path.join(combined_res_csv_path, file)
    print(f'* {file}')

    if os.path.exists(filename):
        Pi = pd.read_csv(filename)

    else:
        Pi = load_upgrade(up, result_path)
        Pi = check_and_correct_for_missing_buildings(Pi, res, up)

        ## (3) save
        Pi.to_csv(filename, index=False)
    display(Pi)
    
print(f'\n>> file saved to: {combined_res_csv_path}')

packages without partitions: 1
* package06.csv


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (58) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_maximum_daily_use_overlap_kw,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw
0,1,136,2021-02-14 11:16:43,2021-02-14 11:19:28,Success,True,Utility Incentive Shell + LEDs,NaN,True,40.0,...,17.067387,7.928824,5.583252,4.838073,15.727273,17.636364,33.961762,24.885217,4793.0,38.521395
1,2,118,2021-02-14 14:31:38,2021-02-14 14:48:43,Success,True,Utility Incentive Shell + LEDs,NaN,True,180.0,...,73.377914,30.915225,16.567100,15.729942,16.181818,15.181818,143.867770,96.809688,5128.0,178.530795
2,3,59,2021-02-14 20:46:03,2021-02-14 20:50:45,Success,True,Utility Incentive Shell + LEDs,NaN,True,80.0,...,46.692477,18.685312,7.409202,7.451741,15.727273,17.363636,94.457778,77.223798,6471.0,101.250343
3,4,13,2021-02-14 17:43:08,2021-02-14 17:45:51,Success,True,Utility Incentive Shell + LEDs,NaN,True,40.0,...,12.428519,6.126783,2.914408,2.974963,15.909091,17.909091,22.183965,13.612700,4932.0,24.757517
4,5,157,2021-02-14 12:10:40,2021-02-14 12:32:37,Success,True,Utility Incentive Shell + LEDs,NaN,True,240.0,...,159.975857,34.427093,117.552666,22.532197,14.272727,12.181818,250.902538,573.700110,619.0,742.351588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,30,2021-02-14 14:08:07,2021-02-14 14:17:01,Success,True,Utility Incentive Shell + LEDs,NaN,True,120.0,...,86.145578,15.070291,50.685497,7.433638,15.090909,11.000000,212.515809,287.954580,873.0,387.854856
20755,20756,105,2021-02-14 15:40:37,2021-02-14 16:28:30,Success,True,Utility Incentive Shell + LEDs,NaN,True,360.0,...,475.096257,81.903933,183.077885,87.340873,12.363636,14.636364,649.285466,934.479949,641.0,1058.176853
20756,20757,34,2021-02-14 20:15:13,2021-02-14 20:31:05,Success,True,Utility Incentive Shell + LEDs,NaN,True,180.0,...,417.117987,43.110124,46.793451,40.667779,13.818182,13.545455,753.541284,665.716358,5103.0,883.479434
20757,20758,124,2021-02-15 01:32:31,2021-02-15 01:44:51,Success,True,Utility Incentive Shell + LEDs,NaN,True,180.0,...,115.612802,22.108526,30.523141,12.564576,14.636364,6.000000,166.187729,702.294224,174.0,806.232685



>> file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_mf/processed results/raw combined csvs


#### (2) Packages WITH single-level partitions (N=10, pkg 07-16)

In [10]:
# 2 partitiions over 'Attic Insulation'

def combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades,
                                           MSHP_option=None, nullify_total_ng=False, save_as_csv=True, 
                                           recreate_from_scratch=True):
    global res
    """
    ARGS:
        pkg_no (int): package number to assign to combined df
        upgrade_list (list): list of ResStock upgrades to combine
        partition_para (str): para to filter the upgrades by and combine
        options_list_for_first_upgrades (list of list): list of para options to filter the first n-1 upgrades by
        MSHP_option (int): option number in package to correct, default to None
        nullify_total_ng (bool): default to False, set total NG use to 0, for electrification package only
        save_as_csv (bool): default to True
        recreate_from_scratch (bool): default to True
    
    RETURN:
        P1: combined df
    """
    file = f'package{pkg_no:02d}.csv'
    filename = os.path.join(combined_res_csv_path, file)
    print(f'* {file}, from upgrades: {upgrade_list}')
    
    N = len(upgrade_list)
    partition_para = f'build_existing_model.{partition_para}'
    
    # check setting:
    if N-1 != len(options_list_for_first_upgrades):
        raise ValueError(f'The size of "options_list_for_first_upgrades" = {len(options_list_for_first_upgrades)}, '+
                         f'is not 1 less than the size of "upgrade_list" = {N}')

    if not os.path.exists(filename) or recreate_from_scratch:
        print(f'building from resstock results... partitioned by:\n  "{partition_para}"')

        ## (1) building ids for each partitions based on partition_para and para_options
        bldg_list = options_list = []; msg = ' '; n_bldgs = 0
        for n in range(N-1):
            bldgs_for_part_n = res[res[partition_para].isin(options_list_for_first_upgrades[n])]['building_id']
            bldg_list.append(bldgs_for_part_n)
            
            options_for_part_n = options_list_for_first_upgrades[n]
            options_list = options_list + options_for_part_n
            
            n_bldgs += len(bldgs_for_part_n)
            msg += f' part {n+1}: {len(bldgs_for_part_n)},'
        
        # for last partition:
        bldgs_for_part_n = res[~res[partition_para].isin(options_list)]['building_id']
        bldg_list.append(bldgs_for_part_n)
        
        n_bldgs += len(bldgs_for_part_n)
        msg += f' part {N}: {len(bldgs_for_part_n)}, total: {n_bldgs}'
        print(msg)

        ## (2) combine partitions and update upgrade name
        P1 = []
        for up, Bi in zip(upgrade_list, bldg_list):
            p = load_upgrade(up, result_path)
            p = check_and_correct_for_missing_buildings(p, res, up)
            P1.append(p[p['building_id'].isin(Bi)])

        P1 = pd.concat(P1, axis=0).sort_index()

        P1['apply_upgrade.upgrade_part'] = P1['apply_upgrade.upgrade_name'] # new col to show partition #
        P1['apply_upgrade.upgrade_name'] = P1['apply_upgrade.upgrade_name'].apply(
            lambda x: ' '.join(str(x).split(' ')[:-3])) # update name
        
        ## (3) correct MSHP costs
        if not MSHP_option == None:
            print('\nrecalculating MSHP cost...')
            MSHP_cost = f'simulation_output_report.option_{MSHP_option:02d}_cost_usd'
            
            # rename orig upgrade_cost col
            if not 'simulation_output_report.upgrade_cost_usd_orig' in P1.columns:
                P1['simulation_output_report.upgrade_cost_usd_orig'] = P1['simulation_output_report.upgrade_cost_usd']

            # copy upgrade_cost col
            P1['simulation_output_report.upgrade_cost_usd'] = P1['simulation_output_report.upgrade_cost_usd_orig']

            # remove old MSHP cost
            P1['simulation_output_report.upgrade_cost_usd'] -= P1[MSHP_cost]

            # recalc upgrade costs for MSHP
            idx = (P1['simulation_output_report.applicable']==True) & (res['build_existing_model.hvac_has_ducts']=='No')
            P1.loc[idx, MSHP_cost] = \
                710 + (95+1800/12)*P1.loc[idx, 'simulation_output_report.size_heating_system_kbtu_h']

            # calculate backup heating system size and cost
            supp_heat_cost = 38 # <--- 38(avg) 28-47 [$/kBtu_h]
            P1['simulation_output_report.size_heating_supp_system_kbtu_h_mshp'] = np.nan
            P1.loc[idx, 'simulation_output_report.size_heating_supp_system_kbtu_h_mshp'] = \
                P1.loc[idx, 'simulation_output_report.hvac_heating_supp_capacity_w']*3.412142/1000

            P1['simulation_output_report.upgrade_cost_usd_supp_heat_mshp'] = np.nan
            P1.loc[idx, 'simulation_output_report.upgrade_cost_usd_supp_heat_mshp'] = \
                supp_heat_cost * P1.loc[idx, 'simulation_output_report.size_heating_supp_system_kbtu_h_mshp']

            # add new MSHP and backup heat cost to 'upgrade_cost_usd'
            P1.loc[idx, 'simulation_output_report.upgrade_cost_usd'] += \
                P1.loc[idx, MSHP_cost]

            P1.loc[idx, 'simulation_output_report.upgrade_cost_usd'] += \
                P1.loc[idx, 'simulation_output_report.upgrade_cost_usd_supp_heat_mshp']

            # show
            print(f'>> package={pkg_no:02d} modified cols:')
            display(P1.loc[idx, ['apply_upgrade.upgrade_name',
                                'simulation_output_report.upgrade_cost_usd_orig',
                                'simulation_output_report.upgrade_cost_usd', # updated
                                'simulation_output_report.upgrade_cost_usd_supp_heat_mshp', # new
                                'simulation_output_report.size_heating_supp_system_kbtu_h_mshp', # new
                               ]])
        
        ## (4) manually set total NG therm to 0 if pkg is electrification
        if nullify_total_ng:
            P1.loc[P1['simulation_output_report.applicable']==True,
                   'simulation_output_report.total_site_natural_gas_therm'] = 0
        
        ## (5) save
        if save_as_csv:
            print(f'\n>> combined file saved to: {combined_res_csv_path}')
            P1.to_csv(filename, index=False)
            
    else:
        P1 = pd.read_csv(filename)

#     display(P1)
    return P1

print('func "combine_df_from_two_partitions"')


func "combine_df_from_two_partitions"


In [11]:
pkg_no = 7
upgrade_list = [2, 3]
partition_para = 'insulation_unfinished_attic'
options_list_for_first_upgrades = [
    ['Uninsulated, Vented','Ceiling R-7, Vented','Ceiling R-13, Vented'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)


* package07.csv, from upgrades: [2, 3]
building from resstock results... partitioned by:
  "build_existing_model.insulation_unfinished_attic"
  part 1: 0, part 2: 20759, total: 20759

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_mf/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,89,2021-02-14 22:48:35,2021-02-14 22:52:07,Success,True,Utility Incentive Shell + LEDs - Forced Air,,True,40.0,...,7.926903,5.610374,4.836418,15.727273,17.636364,33.973656,24.920763,4793.0,38.521327,Utility Incentive Shell + LEDs - Forced Air - ...
1,2,67,2021-02-14 19:31:05,2021-02-14 19:48:47,Success,True,Utility Incentive Shell + LEDs - Forced Air,,True,180.0,...,30.907196,16.697650,15.735254,16.181818,15.181818,143.913959,96.958179,5128.0,178.595872,Utility Incentive Shell + LEDs - Forced Air - ...
2,3,49,2021-02-14 18:35:49,2021-02-14 18:41:06,Success,True,Utility Incentive Shell + LEDs - Forced Air,,True,80.0,...,18.688626,7.425535,7.452862,15.727273,18.181818,94.457822,78.022566,6471.0,101.250336,Utility Incentive Shell + LEDs - Forced Air - ...
3,4,2,2021-02-15 00:29:11,2021-02-15 00:32:43,Success,True,Utility Incentive Shell + LEDs - Forced Air,,True,40.0,...,6.127615,2.925453,2.975776,15.909091,17.909091,22.184124,13.623684,4932.0,24.757670,Utility Incentive Shell + LEDs - Forced Air - ...
4,5,131,2021-02-14 21:18:57,2021-02-14 21:19:43,Invalid,True,Utility Incentive Shell + LEDs - Forced Air,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs - Forced Air - ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,104,2021-02-14 08:41:56,2021-02-14 08:42:40,Invalid,True,Utility Incentive Shell + LEDs - Forced Air,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs - Forced Air - ...
20755,20756,221,2021-02-15 00:09:12,2021-02-15 00:10:06,Invalid,True,Utility Incentive Shell + LEDs - Forced Air,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs - Forced Air - ...
20756,20757,67,2021-02-14 11:38:46,2021-02-14 11:55:28,Success,True,Utility Incentive Shell + LEDs - Forced Air,,True,180.0,...,43.110445,47.137363,40.667986,13.818182,13.545455,753.541855,666.445965,5103.0,883.486797,Utility Incentive Shell + LEDs - Forced Air - ...
20757,20758,146,2021-02-14 16:05:57,2021-02-14 16:06:49,Invalid,True,Utility Incentive Shell + LEDs - Forced Air,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs - Forced Air - ...


In [12]:
pkg_no = 8
upgrade_list = [4, 5]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)

* package08.csv, from upgrades: [4, 5]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 19254, part 2: 1505, total: 20759

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_mf/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,237,2021-02-14 15:17:24,2021-02-14 15:21:44,Success,True,Utility Incentive Shell + LEDs + Appliances - ...,,True,40.0,...,7.542604,5.387337,4.802371,15.727273,17.636364,32.976173,24.624491,4793.0,38.430811,Utility Incentive Shell + LEDs + Appliances - ...
1,2,182,2021-02-14 09:14:58,2021-02-14 09:34:12,Success,True,Utility Incentive Shell + LEDs + Appliances - ...,,True,180.0,...,27.431713,15.797637,14.445226,16.181818,15.181818,136.105123,95.630622,5128.0,171.890025,Utility Incentive Shell + LEDs + Appliances - ...
2,3,121,2021-02-14 14:23:09,2021-02-14 14:29:58,Success,True,Utility Incentive Shell + LEDs + Appliances - ...,,True,80.0,...,16.375229,7.141457,7.315508,15.727273,17.363636,81.349430,68.360159,6471.0,88.900834,Utility Incentive Shell + LEDs + Appliances - ...
3,4,227,2021-02-14 16:23:13,2021-02-14 16:27:33,Success,True,Utility Incentive Shell + LEDs + Appliances - ...,,True,40.0,...,5.715226,2.840710,2.815037,15.636364,17.909091,22.004024,13.419261,4932.0,24.206605,Utility Incentive Shell + LEDs + Appliances - ...
4,5,161,2021-02-14 08:03:50,2021-02-14 08:04:38,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,126,2021-02-14 21:59:48,2021-02-14 22:00:37,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
20755,20756,219,2021-02-15 00:04:07,2021-02-15 00:05:02,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
20756,20757,167,2021-02-14 18:05:03,2021-02-14 18:23:52,Success,True,Utility Incentive Shell + LEDs + Appliances - ...,,True,180.0,...,40.496553,44.111245,39.390061,13.818182,14.000000,564.194450,486.499426,5103.0,696.065021,Utility Incentive Shell + LEDs + Appliances - ...
20757,20758,234,2021-02-14 11:12:02,2021-02-14 11:12:55,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...


In [13]:
pkg_no = 9
upgrade_list = [6, 7]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)


* package09.csv, from upgrades: [6, 7]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 19254, part 2: 1505, total: 20759

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_mf/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,90,2021-02-14 23:12:00,2021-02-14 23:12:45,Invalid,True,Utility Incentive Shell + WH - Hydronic,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + WH - Hydronic - Part 1
1,2,129,2021-02-14 21:30:21,2021-02-14 21:31:04,Invalid,True,Utility Incentive Shell + WH - Hydronic,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + WH - Hydronic - Part 1
2,3,226,2021-02-14 15:31:56,2021-02-14 15:32:49,Invalid,True,Utility Incentive Shell + WH - Hydronic,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + WH - Hydronic - Part 1
3,4,64,2021-02-14 09:09:07,2021-02-14 09:09:47,Invalid,True,Utility Incentive Shell + WH - Hydronic,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + WH - Hydronic - Part 1
4,5,72,2021-02-14 08:49:52,2021-02-14 08:50:38,Invalid,True,Utility Incentive Shell + WH - Hydronic,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + WH - Hydronic - Part 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,81,2021-02-14 21:13:37,2021-02-14 21:14:25,Invalid,True,Utility Incentive Shell + WH - Hydronic,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + WH - Hydronic - Part 1
20755,20756,190,2021-02-15 00:35:31,2021-02-15 00:36:25,Invalid,True,Utility Incentive Shell + WH - Hydronic,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + WH - Hydronic - Part 1
20756,20757,32,2021-02-14 23:18:00,2021-02-14 23:18:48,Invalid,True,Utility Incentive Shell + WH - Hydronic,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + WH - Hydronic - Part 1
20757,20758,191,2021-02-14 23:49:24,2021-02-14 23:50:24,Invalid,True,Utility Incentive Shell + WH - Hydronic,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + WH - Hydronic - Part 1


In [14]:
pkg_no = 10
upgrade_list = [8, 9]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)


* package10.csv, from upgrades: [8, 9]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 19254, part 2: 1505, total: 20759

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_mf/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,207,2021-02-14 17:30:53,2021-02-14 17:31:39,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
1,2,213,2021-02-14 10:36:34,2021-02-14 10:37:20,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
2,3,143,2021-02-14 19:02:04,2021-02-14 19:02:58,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
3,4,169,2021-02-14 16:05:15,2021-02-14 16:05:56,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
4,5,84,2021-02-14 12:37:17,2021-02-14 12:38:04,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,96,2021-02-15 00:53:03,2021-02-15 00:53:50,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
20755,20756,171,2021-02-14 13:00:08,2021-02-14 13:01:06,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
20756,20757,199,2021-02-14 14:06:21,2021-02-14 14:07:12,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
20757,20758,150,2021-02-14 18:13:55,2021-02-14 18:14:49,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...


In [15]:
pkg_no = 11
upgrade_list = [10, 11]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)

* package11.csv, from upgrades: [10, 11]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 19254, part 2: 1505, total: 20759

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_mf/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,69,2021-02-14 19:50:40,2021-02-14 19:55:09,Success,True,Comprehensive - Forced Air + WH + Shell,,True,40.0,...,7.706734,5.412711,4.946413,15.727273,17.818182,32.965476,24.689428,4793.0,38.541800,Comprehensive - Forced Air + WH + Shell - Part 1
1,2,16,2021-02-14 18:53:32,2021-02-14 19:11:41,Success,True,Comprehensive - Forced Air + WH + Shell,,True,180.0,...,29.313382,16.187061,16.104305,16.545455,15.181818,135.373307,96.517946,5128.0,170.822784,Comprehensive - Forced Air + WH + Shell - Part 1
2,3,149,2021-02-15 00:32:58,2021-02-15 00:39:20,Success,True,Comprehensive - Forced Air + WH + Shell,,True,80.0,...,16.202583,7.168497,7.729696,15.727273,17.363636,86.777937,76.425212,857.0,90.340219,Comprehensive - Forced Air + WH + Shell - Part 1
3,4,27,2021-02-14 09:38:22,2021-02-14 09:42:47,Success,True,Comprehensive - Forced Air + WH + Shell,,True,40.0,...,6.006429,2.881312,3.083276,16.818182,17.909091,21.957491,13.525778,4932.0,23.926966,Comprehensive - Forced Air + WH + Shell - Part 1
4,5,160,2021-02-14 12:05:29,2021-02-14 12:06:16,Invalid,True,Comprehensive - Forced Air + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Shell - Part 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,176,2021-02-14 17:13:22,2021-02-14 17:14:08,Invalid,True,Comprehensive - Forced Air + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Shell - Part 1
20755,20756,127,2021-02-14 22:30:25,2021-02-14 22:31:18,Invalid,True,Comprehensive - Forced Air + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Shell - Part 1
20756,20757,166,2021-02-14 20:23:33,2021-02-14 20:41:13,Success,True,Comprehensive - Forced Air + WH + Shell,,True,180.0,...,43.276127,45.590503,41.894195,13.818182,13.545455,745.777733,663.547993,5103.0,878.211150,Comprehensive - Forced Air + WH + Shell - Part 1
20757,20758,122,2021-02-14 19:38:53,2021-02-14 19:39:47,Invalid,True,Comprehensive - Forced Air + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Shell - Part 1


In [16]:
pkg_no = 12
upgrade_list = [12, 13, 14]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50'],
    ['10 ACH50','8 ACH50','7 ACH50','6 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)

* package12.csv, from upgrades: [12, 13, 14]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 17168, part 2: 2992, part 3: 599, total: 20759

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_mf/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,140,2021-02-14 22:14:05,2021-02-14 22:18:10,Success,True,Comprehensive - Forced Air + WH + Insulation +...,,True,40.0,...,7.649822,5.457929,4.894114,15.727273,17.636364,33.103946,24.716112,4793.0,38.543753,Comprehensive - Forced Air + WH + Insulation +...
1,2,87,2021-02-14 16:21:00,2021-02-14 16:39:19,Success,True,Comprehensive - Forced Air + WH + Insulation +...,,True,180.0,...,28.728716,16.630867,15.509109,16.181818,15.181818,137.752085,96.665581,5128.0,173.488983,Comprehensive - Forced Air + WH + Insulation +...
2,3,194,2021-02-14 15:12:49,2021-02-14 15:19:23,Success,True,Comprehensive - Forced Air + WH + Insulation +...,,True,80.0,...,16.540234,7.293125,7.510887,14.181818,18.181818,65.870308,51.537211,4712.0,76.820405,Comprehensive - Forced Air + WH + Insulation +...
3,4,210,2021-02-14 17:55:37,2021-02-14 17:59:33,Success,True,Comprehensive - Forced Air + WH + Insulation +...,,True,40.0,...,5.897427,2.947893,2.961651,15.636364,17.909091,22.182245,13.584627,4932.0,24.408769,Comprehensive - Forced Air + WH + Insulation +...
4,5,38,2021-02-14 16:31:34,2021-02-14 16:32:20,Invalid,True,Comprehensive - Forced Air + WH + Insulation +...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Insulation +...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,208,2021-02-14 10:24:34,2021-02-14 10:25:20,Invalid,True,Comprehensive - Forced Air + WH + Insulation +...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Insulation +...
20755,20756,199,2021-02-14 18:19:24,2021-02-14 18:20:19,Invalid,True,Comprehensive - Forced Air + WH + Insulation +...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Insulation +...
20756,20757,183,2021-02-14 12:25:14,2021-02-14 12:44:23,Success,True,Comprehensive - Forced Air + WH + Insulation +...,,True,180.0,...,43.279794,46.393464,41.883425,13.909091,15.181818,443.147622,370.777319,5103.0,568.609028,Comprehensive - Forced Air + WH + Insulation +...
20757,20758,223,2021-02-14 23:31:37,2021-02-14 23:32:29,Invalid,True,Comprehensive - Forced Air + WH + Insulation +...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Insulation +...


In [17]:
pkg_no = 13
upgrade_list = [15, 16]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)

* package13.csv, from upgrades: [15, 16]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 19254, part 2: 1505, total: 20759

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_mf/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,45,2021-02-14 20:12:47,2021-02-14 20:13:29,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 1
1,2,228,2021-02-15 00:22:49,2021-02-15 00:23:32,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 1
2,3,89,2021-02-14 23:42:10,2021-02-14 23:43:02,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 1
3,4,184,2021-02-14 16:19:41,2021-02-14 16:20:23,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 1
4,5,128,2021-02-14 11:50:35,2021-02-14 11:51:22,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,51,2021-02-14 09:14:58,2021-02-14 09:15:50,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 1
20755,20756,119,2021-02-15 00:33:06,2021-02-15 00:34:06,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 1
20756,20757,112,2021-02-14 14:25:23,2021-02-14 14:26:11,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 1
20757,20758,126,2021-02-14 12:38:06,2021-02-14 12:39:00,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 1


In [18]:
pkg_no = 14
upgrade_list = [17, 18, 19, 20]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50'],
    ['20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
    ['7 ACH50','6 ACH50','5 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)



* package14.csv, from upgrades: [17, 18, 19, 20]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 9661, part 2: 9593, part 3: 1204, part 4: 301, total: 20759

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_mf/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,74,2021-02-14 08:46:48,2021-02-14 08:47:30,Invalid,True,Comprehensive - Hydronic + WH + Shell + Applia...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell + Applia...
1,2,15,2021-02-14 11:15:39,2021-02-14 11:16:21,Invalid,True,Comprehensive - Hydronic + WH + Shell + Applia...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell + Applia...
2,3,238,2021-02-15 02:20:48,2021-02-15 02:21:41,Invalid,True,Comprehensive - Hydronic + WH + Shell + Applia...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell + Applia...
3,4,162,2021-02-14 15:51:34,2021-02-14 15:52:17,Invalid,True,Comprehensive - Hydronic + WH + Shell + Applia...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell + Applia...
4,5,19,2021-02-15 02:13:07,2021-02-15 02:13:55,Invalid,True,Comprehensive - Hydronic + WH + Shell + Applia...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell + Applia...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,57,2021-02-14 10:13:04,2021-02-14 10:13:51,Invalid,True,Comprehensive - Hydronic + WH + Shell + Applia...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell + Applia...
20755,20756,2,2021-02-14 10:34:27,2021-02-14 10:35:25,Invalid,True,Comprehensive - Hydronic + WH + Shell + Applia...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell + Applia...
20756,20757,51,2021-02-14 14:52:09,2021-02-14 14:52:59,Invalid,True,Comprehensive - Hydronic + WH + Shell + Applia...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell + Applia...
20757,20758,171,2021-02-14 16:17:44,2021-02-14 16:18:40,Invalid,True,Comprehensive - Hydronic + WH + Shell + Applia...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell + Applia...


In [19]:
pkg_no = 15
upgrade_list = [21, 22]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)


* package15.csv, from upgrades: [21, 22]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 19254, part 2: 1505, total: 20759

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_mf/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,83,2021-02-14 23:14:35,2021-02-14 23:18:56,Success,True,Comprehensive - ASHP,,True,40.0,...,5.781481,18.235155,4.840785,15.454545,9.272727,32.243241,127.852936,632.0,148.779303,Comprehensive - ASHP - Part 1
1,2,203,2021-02-14 15:51:43,2021-02-14 16:11:38,Success,True,Comprehensive - ASHP,,True,180.0,...,18.174420,42.163104,14.572880,14.090909,10.090909,103.177338,352.906182,630.0,399.055496,Comprehensive - ASHP - Part 1
2,3,41,2021-02-14 23:09:06,2021-02-14 23:15:33,Success,True,Comprehensive - ASHP,,True,80.0,...,8.784858,14.794793,6.360520,15.181818,12.181818,75.512299,259.442499,619.0,308.321659,Comprehensive - ASHP - Part 1
3,4,122,2021-02-14 23:30:12,2021-02-14 23:34:14,Success,True,Comprehensive - ASHP,,True,40.0,...,3.338963,8.114534,2.611518,16.545455,10.181818,16.474158,60.553234,630.0,70.346035,Comprehensive - ASHP - Part 1
4,5,65,2021-02-14 08:04:13,2021-02-14 08:05:02,Invalid,True,Comprehensive - ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - ASHP - Part 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,182,2021-02-14 18:28:43,2021-02-14 18:29:34,Invalid,True,Comprehensive - ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - ASHP - Part 1
20755,20756,140,2021-02-14 13:45:45,2021-02-14 13:46:41,Invalid,True,Comprehensive - ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - ASHP - Part 1
20756,20757,85,2021-02-14 08:32:36,2021-02-14 08:51:47,Success,True,Comprehensive - ASHP,,True,180.0,...,42.679423,68.502770,42.676401,13.818182,9.272727,689.597278,1023.557894,643.0,1311.815244,Comprehensive - ASHP - Part 1
20757,20758,69,2021-02-14 22:01:42,2021-02-14 22:02:38,Invalid,True,Comprehensive - ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - ASHP - Part 1


In [20]:
pkg_no = 16
upgrade_list = [23, 24]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
]
MSHP_option = None #4

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)

* package16.csv, from upgrades: [23, 24]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 19254, part 2: 1505, total: 20759

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_mf/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,98,2021-02-14 09:36:42,2021-02-14 09:37:24,Invalid,True,Comprehensive - MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - MSHP - Part 1
1,2,102,2021-02-15 00:10:19,2021-02-15 00:11:04,Invalid,True,Comprehensive - MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - MSHP - Part 1
2,3,118,2021-02-14 16:53:22,2021-02-14 16:54:17,Invalid,True,Comprehensive - MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - MSHP - Part 1
3,4,6,2021-02-14 20:20:35,2021-02-14 20:21:18,Invalid,True,Comprehensive - MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - MSHP - Part 1
4,5,154,2021-02-15 00:45:01,2021-02-15 00:45:47,Invalid,True,Comprehensive - MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - MSHP - Part 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,216,2021-02-14 12:10:43,2021-02-14 12:11:31,Invalid,True,Comprehensive - MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - MSHP - Part 1
20755,20756,98,2021-02-14 22:20:54,2021-02-14 22:21:49,Invalid,True,Comprehensive - MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - MSHP - Part 1
20756,20757,50,2021-02-15 00:56:16,2021-02-15 00:57:07,Invalid,True,Comprehensive - MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - MSHP - Part 1
20757,20758,236,2021-02-14 08:28:02,2021-02-14 08:28:54,Invalid,True,Comprehensive - MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - MSHP - Part 1


#### (3) Packages WITH multi-level partitions (N=4, pkg 17-20)

In [21]:
def combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option=None, 
                                         fix_upgrade_name=False, nullify_total_ng=False, save_as_csv=True, 
                                         recreate_from_scratch=True):
    global res
    """
    ARGS:
        pkg_no (int): package number to assign to combined df
        upgrade_list (list): list of ResStock upgrades to combine
        level1_list (list of dict): list of dictionaries defining the level 1 partition key and options
        level2_list (list of dict): list of dictionaries defining the level 2 partition key and options
        MSHP_option (int): option number in package to correct, default to None
        fix_upgrade_name (bool): default to True, for pkg 21-24 only
        nullify_total_ng (bool): default to False, set total NG use to 0, for electrification package only
        save_as_csv (bool): default to True
        recreate_from_scratch (bool): default to True
    
    RETURN:
        P1: combined df
    """
    
    file = f'package{pkg_no:02d}.csv'
    filename = os.path.join(combined_res_csv_path, file)
    print(f'* {file}, from upgrades: {upgrade_list}')
    
    N_upgrades = len(upgrade_list)
    N_partitions = len(level1_list)*len(level2_list)
    if N_upgrades != N_partitions:
        raise ValueError(f'The size of "upgrade_list" = {N_upgrades} does not match the number of enumeration from '+
                        f'"level1_list" and "level2_list" = {N_partitions}')
    
    if not os.path.exists(filename) or recreate_from_scratch:
        
        print(f'building from resstock run results, {N_partitions} partitions...\n')

        ## building ids for each partitions
        bldg_list = []; CBi_len = 0

        ##### level 1
        P4C1 = level1_list # <----
        for i , Ci in enumerate(P4C1,1):
            CBi = set()
            for key, lst in Ci.items():
                Bi = res[res[f'build_existing_model.{key}'].replace({np.nan:'None'}).isin(lst)]['building_id']
                CBi = CBi.union(set(list(Bi)))
                print(f'- 1.{i} {key}, {len(Bi)} / {len(res)}')
            print(f'- 1.{i} total, {len(CBi)}')
            res1 = res[res[f'building_id'].isin(CBi)]

            ##### level 2
            P4C2 = level2_list # <----
            CBi_2all = set(); CBi_2len = 0
            for k, Ci in enumerate(P4C2,1):
                if k == len(P4C2):
                    CBi = set(list(res1['building_id'])).difference(CBi_2all)
                    print(f'    + 3.{k} total, {len(CBi)}')
                else:
                    CBi = set()
                    for key, lst in Ci.items():
                        Bi = res1[res1[f'build_existing_model.{key}'].isin(lst)]['building_id']
                        CBi = CBi.union(set(list(Bi)))
                        print(f'    + 3.{k} {key}, {len(Bi)} / {len(res1)}')
                CBi_2all = CBi_2all.union(CBi)
                bldg_list.append(CBi)

                CBi_2len += len(CBi)
                CBi_len += len(CBi)
                print(f'appending total {len(CBi)}, (1.{i})(2.{k})_culm = {CBi_2len}, overall_culm = {CBi_len}\n')

        ## (2) combine partitions, update upgrade name
        P4 = []
        for up, Bi in zip(upgrade_list, bldg_list):
            p = load_upgrade(up, result_path)
            p = check_and_correct_for_missing_buildings(p, res, up)
            P4.append(p[p['building_id'].isin(Bi)])

        P4 = pd.concat(P4, axis=0).sort_index()

        P4['apply_upgrade.upgrade_part'] = P4['apply_upgrade.upgrade_name'] # new col to show partition #
        P4['apply_upgrade.upgrade_name'] = P4['apply_upgrade.upgrade_name'].apply(
            lambda x: ' '.join(str(x).split(' ')[:-3])) # update name
        
        ## (3) correct MSHP costs
        if not MSHP_option == None:
            print('\nrecalculating MSHP cost...')
            MSHP_cost = f'simulation_output_report.option_{MSHP_option:02d}_cost_usd'
            
            # rename orig upgrade_cost col
            if not 'simulation_output_report.upgrade_cost_usd_orig' in P4.columns:
                P4['simulation_output_report.upgrade_cost_usd_orig'] = P4['simulation_output_report.upgrade_cost_usd']

            # copy upgrade_cost col
            P4['simulation_output_report.upgrade_cost_usd'] = P4['simulation_output_report.upgrade_cost_usd_orig']

            # remove old MSHP cost
            P4['simulation_output_report.upgrade_cost_usd'] -= P4[MSHP_cost]

            # recalc upgrade costs for MSHP
            idx = (P4['simulation_output_report.applicable']==True) & (res['build_existing_model.hvac_has_ducts']=='No')
            P4.loc[idx, MSHP_cost] = \
                710 + (95+1800/12)*P4.loc[idx, 'simulation_output_report.size_heating_system_kbtu_h']

            # calculate backup heating system size and cost
            supp_heat_cost = 38 # <--- 38(avg) 28-47 [$/kBtu_h]
            P4['simulation_output_report.size_heating_supp_system_kbtu_h_mshp'] = np.nan
            P4.loc[idx, 'simulation_output_report.size_heating_supp_system_kbtu_h_mshp'] = \
                P4.loc[idx, 'simulation_output_report.hvac_heating_supp_capacity_w']*3.412142/1000

            P4['simulation_output_report.upgrade_cost_usd_supp_heat_mshp'] = np.nan
            P4.loc[idx, 'simulation_output_report.upgrade_cost_usd_supp_heat_mshp'] = \
                supp_heat_cost * P4.loc[idx, 'simulation_output_report.size_heating_supp_system_kbtu_h_mshp']

            # add new MSHP and backup heat cost to 'upgrade_cost_usd'
            P4.loc[idx, 'simulation_output_report.upgrade_cost_usd'] += \
                P4.loc[idx, MSHP_cost]

            P4.loc[idx, 'simulation_output_report.upgrade_cost_usd'] += \
                P4.loc[idx, 'simulation_output_report.upgrade_cost_usd_supp_heat_mshp']

            # show
            print(f'>> package={pkg_no:02d} modified cols:')
            display(P4.loc[idx, ['apply_upgrade.upgrade_name',
                                'simulation_output_report.upgrade_cost_usd_orig',
                                'simulation_output_report.upgrade_cost_usd', # updated
                                'simulation_output_report.upgrade_cost_usd_supp_heat_mshp', # new
                                'simulation_output_report.size_heating_supp_system_kbtu_h_mshp', # new
                               ]])
            
        ## (4) fix upgrade name
        try:
            P4[['apply_upgrade.upgrade_name','apply_upgrade.upgrade_part']] = \
                P4[['apply_upgrade.upgrade_name','apply_upgrade.upgrade_part']].apply(
                lambda x: x.str.replace('low-E Window','Low-Gain Window')) # fix upgrade name
        except:
            pass
        
        ## (5) 
        if nullify_total_ng:
            P4.loc[P4['simulation_output_report.applicable']==True,
                   'simulation_output_report.total_site_natural_gas_therm'] = 0

        ## (6) save
        P4.to_csv(filename, index=False)

    else:              
        P4 = pd.read_csv(filename)
    
    return P4

print('func "combine_df_from_two_level_partitions" loaded')

func "combine_df_from_two_level_partitions" loaded


In [22]:
pkg_no = 17 # <----
upgrade_list = list(range(25, 25+8)) # <----
MSHP_option = None 
fix_upgrade_name = False
    
## (1) get partition keys and options
# para 1 -- 4 parts #######
key = 'infiltration'
PC = [
    {key: ['50 ACH50','40 ACH50','30 ACH50','25 ACH50']},
    {key: ['20 ACH50','15 ACH50','10 ACH50','8 ACH50']},
    {key: ['7 ACH50','6 ACH50','5 ACH50']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level1_list = PC.copy() # <----

# para 2 -- 2 parts #######
key = 'insulation_unfinished_attic'
PC = [
    {key: ['Uninsulated, Vented', 'Ceiling R-7, Vented', 'Ceiling R-13, Vented']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level2_list = PC.copy() # <----
    
# (2) create df
combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option,
                                     fix_upgrade_name, nullify_total_ng=True, save_as_csv=True, 
                                     recreate_from_scratch=True)



* package17.csv, from upgrades: [25, 26, 27, 28, 29, 30, 31, 32]
building from resstock run results, 8 partitions...

- 1.1 infiltration, 9661 / 20759
- 1.1 total, 9661
    + 3.1 insulation_unfinished_attic, 0 / 9661
appending total 0, (1.1)(2.1)_culm = 0, overall_culm = 0

    + 3.2 total, 9661
appending total 9661, (1.1)(2.2)_culm = 9661, overall_culm = 9661

- 1.2 infiltration, 9593 / 20759
- 1.2 total, 9593
    + 3.1 insulation_unfinished_attic, 0 / 9593
appending total 0, (1.2)(2.1)_culm = 0, overall_culm = 9661

    + 3.2 total, 9593
appending total 9593, (1.2)(2.2)_culm = 9593, overall_culm = 19254

- 1.3 infiltration, 1204 / 20759
- 1.3 total, 1204
    + 3.1 insulation_unfinished_attic, 0 / 1204
appending total 0, (1.3)(2.1)_culm = 0, overall_culm = 19254

    + 3.2 total, 1204
appending total 1204, (1.3)(2.2)_culm = 1204, overall_culm = 20458

- 1.4 infiltration, 301 / 20759
- 1.4 total, 301
    + 3.1 insulation_unfinished_attic, 0 / 301
appending total 0, (1.4)(2.1)_culm = 0,

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,61,2021-02-14 14:44:19,2021-02-14 14:49:52,Success,True,Electrification - Gas to ASHP,,True,40.0,...,5.905790,19.288311,5.090518,13.636364,15.000000,50.154549,139.399021,894.0,164.033454,Electrification - Gas to ASHP - Part 2
1,2,5,2021-02-14 21:29:35,2021-02-14 21:52:02,Success,True,Electrification - Gas to ASHP,,True,180.0,...,18.767640,44.944814,15.052927,14.090909,9.454545,117.041424,354.874606,624.0,389.667711,Electrification - Gas to ASHP - Part 4
2,3,63,2021-02-14 09:28:40,2021-02-14 09:37:46,Success,True,Electrification - Gas to ASHP,,True,80.0,...,9.057049,16.322663,6.816327,14.545455,13.909091,59.931183,255.094729,175.0,318.613243,Electrification - Gas to ASHP - Part 4
3,4,110,2021-02-14 22:06:29,2021-02-14 22:12:52,Success,True,Electrification - Gas to ASHP,,True,40.0,...,3.460046,8.930444,2.739169,15.727273,11.000000,20.699757,66.179774,895.0,95.991298,Electrification - Gas to ASHP - Part 4
4,5,130,2021-02-14 16:07:45,2021-02-14 16:08:31,Invalid,True,Electrification - Gas to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to ASHP - Part 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,87,2021-02-14 19:34:44,2021-02-14 19:35:30,Invalid,True,Electrification - Gas to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to ASHP - Part 2
20755,20756,225,2021-02-14 20:32:47,2021-02-14 20:33:43,Invalid,True,Electrification - Gas to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to ASHP - Part 4
20756,20757,173,2021-02-14 16:13:49,2021-02-14 16:38:07,Success,True,Electrification - Gas to ASHP,,True,180.0,...,42.813882,87.105715,43.473349,12.090909,11.636364,514.333684,1127.563168,703.0,1465.360867,Electrification - Gas to ASHP - Part 4
20757,20758,127,2021-02-14 21:26:52,2021-02-14 21:27:43,Invalid,True,Electrification - Gas to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to ASHP - Part 2


In [23]:
pkg_no = 18 # <----
upgrade_list = list(range(33, 33+12)) # <----
MSHP_option = None #5 # <----
fix_upgrade_name = False

## (1) get partition keys and options
# para 1 -- 4 parts  #######
key = 'infiltration'
PC = [
    {key: ['50 ACH50','40 ACH50','30 ACH50','25 ACH50']},
    {key: ['20 ACH50','15 ACH50','10 ACH50','8 ACH50']},
    {key: ['7 ACH50','6 ACH50','5 ACH50']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level1_list = PC.copy() # <----

# para 2 -- 3 parts #######
key = 'insulation_unfinished_attic'
PC = [
    {key: ['Uninsulated, Vented', 'Ceiling R-7, Vented']},
    {key: ['Ceiling R-13, Vented', 'Ceiling R-19, Vented']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level2_list = PC.copy() # <----
    
# (2) create df
combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option,
                                     fix_upgrade_name, nullify_total_ng=True, save_as_csv=True, 
                                     recreate_from_scratch=True)


* package18.csv, from upgrades: [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]
building from resstock run results, 12 partitions...

- 1.1 infiltration, 9661 / 20759
- 1.1 total, 9661
    + 3.1 insulation_unfinished_attic, 0 / 9661
appending total 0, (1.1)(2.1)_culm = 0, overall_culm = 0

    + 3.2 insulation_unfinished_attic, 0 / 9661
appending total 0, (1.1)(2.2)_culm = 0, overall_culm = 0

    + 3.3 total, 9661
appending total 9661, (1.1)(2.3)_culm = 9661, overall_culm = 9661

- 1.2 infiltration, 9593 / 20759
- 1.2 total, 9593
    + 3.1 insulation_unfinished_attic, 0 / 9593
appending total 0, (1.2)(2.1)_culm = 0, overall_culm = 9661

    + 3.2 insulation_unfinished_attic, 0 / 9593
appending total 0, (1.2)(2.2)_culm = 0, overall_culm = 9661

    + 3.3 total, 9593
appending total 9593, (1.2)(2.3)_culm = 9593, overall_culm = 19254

- 1.3 infiltration, 1204 / 20759
- 1.3 total, 1204
    + 3.1 insulation_unfinished_attic, 0 / 1204
appending total 0, (1.3)(2.1)_culm = 0, overall_culm = 

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,113,2021-02-14 19:20:25,2021-02-14 19:21:05,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 3
1,2,127,2021-02-14 11:50:33,2021-02-14 11:51:17,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 6
2,3,119,2021-02-14 17:31:12,2021-02-14 17:32:02,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 6
3,4,94,2021-02-14 19:16:56,2021-02-14 19:17:38,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 6
4,5,231,2021-02-14 14:14:15,2021-02-14 14:15:01,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,30,2021-02-14 19:17:13,2021-02-14 19:17:59,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 3
20755,20756,2,2021-02-14 18:54:36,2021-02-14 18:55:32,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 6
20756,20757,168,2021-02-14 19:53:47,2021-02-14 19:54:34,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 6
20757,20758,7,2021-02-14 16:52:57,2021-02-14 16:53:53,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 3


In [24]:
pkg_no = 19 # <----
upgrade_list = list(range(45, 45+8)) # <----
MSHP_option = None
fix_upgrade_name = False

## (1) get partition keys and options
# para 1 -- 4 parts #######
key = 'infiltration'
PC = [
    {key: ['50 ACH50','40 ACH50','30 ACH50','25 ACH50']},
    {key: ['20 ACH50','15 ACH50','10 ACH50','8 ACH50']},
    {key: ['7 ACH50','6 ACH50','5 ACH50']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level1_list = PC.copy() # <----

# para 2 -- 2 parts #######
key = 'insulation_unfinished_attic'
PC = [
    {key: ['Uninsulated, Vented', 'Ceiling R-7, Vented', 'Ceiling R-13, Vented']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level2_list = PC.copy() # <----
    
# (2) create df
combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option,
                                     fix_upgrade_name, nullify_total_ng=True, save_as_csv=True, 
                                     recreate_from_scratch=True)


* package19.csv, from upgrades: [45, 46, 47, 48, 49, 50, 51, 52]
building from resstock run results, 8 partitions...

- 1.1 infiltration, 9661 / 20759
- 1.1 total, 9661
    + 3.1 insulation_unfinished_attic, 0 / 9661
appending total 0, (1.1)(2.1)_culm = 0, overall_culm = 0

    + 3.2 total, 9661
appending total 9661, (1.1)(2.2)_culm = 9661, overall_culm = 9661

- 1.2 infiltration, 9593 / 20759
- 1.2 total, 9593
    + 3.1 insulation_unfinished_attic, 0 / 9593
appending total 0, (1.2)(2.1)_culm = 0, overall_culm = 9661

    + 3.2 total, 9593
appending total 9593, (1.2)(2.2)_culm = 9593, overall_culm = 19254

- 1.3 infiltration, 1204 / 20759
- 1.3 total, 1204
    + 3.1 insulation_unfinished_attic, 0 / 1204
appending total 0, (1.3)(2.1)_culm = 0, overall_culm = 19254

    + 3.2 total, 1204
appending total 1204, (1.3)(2.2)_culm = 1204, overall_culm = 20458

- 1.4 infiltration, 301 / 20759
- 1.4 total, 301
    + 3.1 insulation_unfinished_attic, 0 / 301
appending total 0, (1.4)(2.1)_culm = 0,

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,51,2021-02-14 15:06:42,2021-02-14 15:07:25,Invalid,True,Electrification - Electric to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP - Part 2
1,2,198,2021-02-14 09:37:15,2021-02-14 09:37:58,Invalid,True,Electrification - Electric to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP - Part 4
2,3,231,2021-02-14 09:55:31,2021-02-14 09:56:21,Invalid,True,Electrification - Electric to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP - Part 4
3,4,147,2021-02-14 08:22:26,2021-02-14 08:23:11,Invalid,True,Electrification - Electric to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP - Part 4
4,5,185,2021-02-14 08:43:42,2021-02-14 09:18:53,Success,True,Electrification - Electric to ASHP,,True,240.0,...,26.861075,58.143479,23.303374,14.272727,12.090909,264.735697,605.124542,618.0,846.658402,Electrification - Electric to ASHP - Part 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,126,2021-02-14 19:31:38,2021-02-14 19:32:24,Invalid,True,Electrification - Electric to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP - Part 2
20755,20756,10,2021-02-14 21:44:25,2021-02-14 21:45:21,Invalid,True,Electrification - Electric to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP - Part 4
20756,20757,109,2021-02-14 08:47:59,2021-02-14 08:48:47,Invalid,True,Electrification - Electric to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP - Part 4
20757,20758,1,2021-02-14 11:50:59,2021-02-14 11:51:52,Invalid,True,Electrification - Electric to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP - Part 2


In [25]:
pkg_no = 20 # <----
upgrade_list = list(range(53, 53+12)) # <----
MSHP_option = None #5 # <----
fix_upgrade_name = False

## (1) get partition keys and options
# para 1 -- 4 parts  #######
key = 'infiltration'
PC = [
    {key: ['50 ACH50','40 ACH50','30 ACH50','25 ACH50']},
    {key: ['20 ACH50','15 ACH50','10 ACH50','8 ACH50']},
    {key: ['7 ACH50','6 ACH50','5 ACH50']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level1_list = PC.copy() # <----

# para 2 -- 3 parts #######
key = 'insulation_unfinished_attic'
PC = [
    {key: ['Uninsulated, Vented', 'Ceiling R-7, Vented']},
    {key: ['Ceiling R-13, Vented', 'Ceiling R-19, Vented']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level2_list = PC.copy() # <----
    
# (2) create df
combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option,
                                     fix_upgrade_name, nullify_total_ng=True, save_as_csv=True, 
                                     recreate_from_scratch=True)



* package20.csv, from upgrades: [53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
building from resstock run results, 12 partitions...

- 1.1 infiltration, 9661 / 20759
- 1.1 total, 9661
    + 3.1 insulation_unfinished_attic, 0 / 9661
appending total 0, (1.1)(2.1)_culm = 0, overall_culm = 0

    + 3.2 insulation_unfinished_attic, 0 / 9661
appending total 0, (1.1)(2.2)_culm = 0, overall_culm = 0

    + 3.3 total, 9661
appending total 9661, (1.1)(2.3)_culm = 9661, overall_culm = 9661

- 1.2 infiltration, 9593 / 20759
- 1.2 total, 9593
    + 3.1 insulation_unfinished_attic, 0 / 9593
appending total 0, (1.2)(2.1)_culm = 0, overall_culm = 9661

    + 3.2 insulation_unfinished_attic, 0 / 9593
appending total 0, (1.2)(2.2)_culm = 0, overall_culm = 9661

    + 3.3 total, 9593
appending total 9593, (1.2)(2.3)_culm = 9593, overall_culm = 19254

- 1.3 infiltration, 1204 / 20759
- 1.3 total, 1204
    + 3.1 insulation_unfinished_attic, 0 / 1204
appending total 0, (1.3)(2.1)_culm = 0, overall_culm = 

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,45,2021-02-14 13:00:17,2021-02-14 13:00:58,Invalid,True,Electrification - Electric to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP - Part 3
1,2,154,2021-02-14 13:34:28,2021-02-14 13:35:12,Invalid,True,Electrification - Electric to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP - Part 6
2,3,49,2021-02-14 22:37:15,2021-02-14 22:38:05,Invalid,True,Electrification - Electric to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP - Part 6
3,4,130,2021-02-15 01:01:23,2021-02-15 01:02:04,Invalid,True,Electrification - Electric to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP - Part 6
4,5,201,2021-02-14 16:23:13,2021-02-14 16:23:59,Invalid,True,Electrification - Electric to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP - Part 3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,128,2021-02-14 18:05:41,2021-02-14 18:21:12,Success,True,Electrification - Electric to MSHP,,True,120.0,...,8.714471,18.554520,7.248738,15.636364,11.545455,102.493966,151.710208,873.0,199.882762,Electrification - Electric to MSHP - Part 3
20755,20756,181,2021-02-14 17:15:39,2021-02-14 18:20:07,Success,True,Electrification - Electric to MSHP,,True,360.0,...,71.333984,111.013790,70.684311,11.000000,12.545455,1126.483323,1365.209185,1519.0,1712.577463,Electrification - Electric to MSHP - Part 6
20756,20757,214,2021-02-14 23:47:46,2021-02-14 23:48:34,Invalid,True,Electrification - Electric to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP - Part 6
20757,20758,131,2021-02-14 18:17:06,2021-02-14 18:40:32,Success,True,Electrification - Electric to MSHP,,True,180.0,...,14.599278,18.174951,11.779881,14.636364,6.000000,110.697389,354.036735,174.0,424.186218,Electrification - Electric to MSHP - Part 3


## 2B. PACKAGE summary ###

In [26]:
### upgrade result processing funcs
def assign_utility_rates_to_upgrade(upkg_no, p, ref, HVAC_upgrades_rate_change, for_packages=False):
    
    res = ref.set_index('building_id')[['gas_rate', 'gas_fixed', 'gas_CO2_rate',
                                        'elec_rate', 'elec_fixed', 'elec_CO2_rate'
                                       ]]
    ## assign rates
    p['gas_rate'] = p['building_id'].map(res['gas_rate'])
    p['gas_fixed'] = p['building_id'].map(res['gas_fixed'])
    p['gas_CO2_rate'] = p['building_id'].map(res['gas_CO2_rate'])
    p['elec_rate'] = p['building_id'].map(res['elec_rate'])
    p['elec_fixed'] = p['building_id'].map(res['elec_fixed'])
    p['elec_CO2_rate'] = p['building_id'].map(res['elec_CO2_rate'])
    
    if not for_packages:
        # ind upgrades
        for n in HVAC_upgrades_rate_change.keys():
            new_rates = HVAC_upgrades_rate_change[n]
            p.loc[p['simulation_output_report.applicable']==True, 'gas_rate'] = new_rates[0]
            p.loc[p['simulation_output_report.applicable']==True, 'gas_fixed'] = new_rates[1]
            p.loc[p['simulation_output_report.applicable']==True, 'elec_rate'] = new_rates[2]
            p.loc[p['simulation_output_report.applicable']==True, 'elec_fixed'] = new_rates[3]
    else:
        if isinstance(HVAC_upgrades_rate_change, dict):
            # packages with a dict input
            if upkg_no in HVAC_upgrades_rate_change.keys():
                for m in HVAC_upgrades_rate_change[upkg_no]:
                    idx = (p[(p['apply_upgrade.applicable']==True) &
                           (~p[f'simulation_output_report.option_{m:02d}_cost_usd'].isnull())
                            ]).index
                    p.loc[idx, 'gas_rate'] = NGH_rate
                    p.loc[idx, 'gas_fixed'] = NGH_fixed
                    p.loc[idx, 'elec_rate'] = EH_rate
                    p.loc[idx, 'elec_fixed'] = EH_fixed
        
        else:
            # packages with a list input
            for m in HVAC_upgrades_rate_change:
                idx = (p[(p['apply_upgrade.applicable']==True) &
                       (~p[f'simulation_output_report.option_{m:02d}_cost_usd'].isnull())
                        ]).index
                p.loc[idx, 'gas_rate'] = NGH_rate
                p.loc[idx, 'gas_fixed'] = NGH_fixed
                p.loc[idx, 'elec_rate'] = EH_rate
                p.loc[idx, 'elec_fixed'] = EH_fixed
                
    # assign 0 rates to building with no energy use  
    p.loc[p['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_fixed'] = 0 
    p.loc[p['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_rate'] = 0 
    p.loc[p['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_fixed'] = 0 
    p.loc[p['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_rate'] = 0 
    
    for col in ['gas_rate','gas_fixed','gas_CO2_rate','elec_rate','elec_fixed','elec_CO2_rate']:
        p.loc[p['simulation_output_report.applicable']==False, col] = np.nan

    return p

def combine_upgrade_cost_and_lifetime(p):
    
    ## upgrade costs (sum)
    p['upgrade_cost'] = p['simulation_output_report.upgrade_cost_usd']
    cost_cols = list(x for x in p.columns if x.endswith('cost_usd'))
    p = p.drop(cost_cols, axis=1)
    
    ## upgrade lifetime (min)
    lt_cols = list(x for x in p.columns if x.endswith('lifetime_yrs'))
    p['upgrade_lifetime'] = p[lt_cols].min(axis=1)
    p = p.drop(lt_cols, axis=1)

    p['upgrade_cost'] = p['upgrade_cost'].replace([0, None,''],np.nan)
    p['upgrade_lifetime'] = p['upgrade_lifetime'].replace([0, None,''],np.nan)
    
    return p

def get_annual_totals(pp, get_col_only=False):
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
        
    p['ann_gas_cost'] = \
        p['simulation_output_report.total_site_natural_gas_therm']*p['gas_rate']+p['gas_fixed']
    p['ann_elec_cost'] = \
        p['simulation_output_report.total_site_electricity_kwh']*p['elec_rate']+p['elec_fixed']
    p['ann_energy_cost'] = \
        p['ann_gas_cost'] + p['ann_elec_cost']
    
    p['ann_metric_ton_co2e_gas'] = \
        p['simulation_output_report.total_site_natural_gas_therm']*p['gas_CO2_rate']
    p['ann_metric_ton_co2e_elec'] = \
        p['simulation_output_report.total_site_electricity_kwh']*p['elec_CO2_rate']
    p['ann_metric_ton_co2e'] = \
        p['ann_metric_ton_co2e_gas'] + p['ann_metric_ton_co2e_elec']
    
    if get_col_only:
        return p[['ann_gas_cost','ann_elec_cost','ann_energy_cost',
                 'ann_metric_ton_co2e_gas','ann_metric_ton_co2e_elec','ann_metric_ton_co2e']]
    else:
        return p

def get_annual_gas_elec_site_energy_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_therm_gas_saving'] = ref['simulation_output_report.total_site_natural_gas_therm']-\
         p['simulation_output_report.total_site_natural_gas_therm']
    p['ann_kwh_elec_saving'] = ref['simulation_output_report.total_site_electricity_kwh']-\
         p['simulation_output_report.total_site_electricity_kwh']
    p['ann_mbtu_site_energy_saving'] = ref['simulation_output_report.total_site_energy_mbtu']-\
        p['simulation_output_report.total_site_energy_mbtu']
    
    if get_col_only:
        return p[['ann_therm_gas_saving','ann_kwh_elec_saving','ann_mbtu_site_energy_saving']]
    else:
        return p

def get_annual_energy_cost_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """ 
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_gas_cost_saving'] = \
        ref['simulation_output_report.total_site_natural_gas_therm']*ref['gas_rate']+ref['gas_fixed'] - \
        (p['simulation_output_report.total_site_natural_gas_therm']*p['gas_rate']+p['gas_fixed'])
        
    p['ann_elec_cost_saving'] = \
        ref['simulation_output_report.total_site_electricity_kwh']*ref['elec_rate']+ref['elec_fixed'] - \
        (p['simulation_output_report.total_site_electricity_kwh']*p['elec_rate']+p['elec_fixed'])
    
    p['ann_energy_cost_saving'] = p['ann_gas_cost_saving']+p['ann_elec_cost_saving'] #p[['ann_gas_cost_saving','ann_elec_cost_saving']].sum(axis=1)
    
    if get_col_only:
        return p['ann_energy_cost_saving']
    else:
        return p

def get_annual_metric_ton_co2e_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_metric_ton_co2e_saving_gas'] = \
        (ref['simulation_output_report.total_site_natural_gas_therm']-\
         p['simulation_output_report.total_site_natural_gas_therm'])*p['gas_CO2_rate']
    
    p['ann_metric_ton_co2e_saving_elec'] = \
        (ref['simulation_output_report.total_site_electricity_kwh']-\
         p['simulation_output_report.total_site_electricity_kwh'])*p['elec_CO2_rate']
    
    p['ann_metric_ton_co2e_saving'] = p['ann_metric_ton_co2e_saving_gas'] + p['ann_metric_ton_co2e_saving_elec']
    
    if get_col_only:
        return p['ann_metric_ton_co2e_saving']
    else:
        return p

def print_metrics_report(p, has_comparative_payback=True):
    ### (1) check for simple_payback > 100 yr or if < 0 yr
    if len(p[~p['simple_payback'].isnull()]):
        print(p['simple_payback'].agg(['min','median','max']))
    spb_100 = p[p['simple_payback']>100]
    if len(spb_100)>0:
        print(f'    *PAYBACK1 - too large* upgrade={n} has {len(spb_100)} simple_payback>100 ' +\
              f'(including {len(p[p["simple_payback"]==np.inf])} INF)')
    spb_neg = p[p['simple_payback']<0]
    if len(spb_neg)>0:
        print(f'    *PAYBACK1 - negative*  upgrade={n} has {len(spb_neg)} simple_payback<0 ' +\
              '(due to negative energy cost saving)')

    ### (2) check for comparative_payback > 100 yr or if < 0 yr
    if has_comparative_payback:
        if len(p[~p['comparative_payback'].isnull()]):
            print(p['comparative_payback'].agg(['min','median','max']))
        spb_100 = p[p['comparative_payback']>100]
        if len(spb_100)>0:
            print(f'    *PAYBACK2 - too large* upgrade={n} has {len(spb_100)} comparative_payback>100 ' +\
                  f'(including {len(p[p["comparative_payback"]==np.inf])} INF)')
        spb_neg = p[p['comparative_payback']<0]
        if len(spb_neg)>0:
            print(f'    *PAYBACK2 - negative*  upgrade={n} has {len(spb_neg)} comparative_payback<0 ' +\
                  '(due to negative energy cost saving)')

    ### (3) check for eui==inf
    for eui in ['pct_delta_gas_eui','pct_delta_elec_eui','pct_delta_site_eui']:
        eui_inf = p[p[eui]==np.inf]
        if len(eui_inf)>0:
            print(f'       *EUI - inf* upgrade={n} has {len(eui_inf)} {eui}=INF ' +\
                  '(due to fuel introduction from upgrade)')

    ### (4) check for neg carbon savings
    ces_neg = p[p['ann_metric_ton_co2e_saving']<0]
    if len(ces_neg)>0:
        print(f'    *CARBON - negative*  upgrade={n} has {len(ces_neg)} carbon saving<0 ')
        
print('funcs loaded')

funcs loaded


In [27]:
### set utility rates ###
NG_rate_multiplier = 1 # <-----

if NG_rate_multiplier > 1:
    fn_ext = f'_{NG_rate_multiplier}x_gas_prices' # file name extension to add to relevant results
else:
    fn_ext = ''

### utility rates ###########################################
# ref (EIA): 
# avg ComEd res elec rate 2019: $ 0.1330 /kWh
# weighted avg IL gas rate 2019: $ 0.77183 /therm

## electricity ##
# annual fixed rates = monthly x 12
EH_fixed = 15.70 * 12 # annual
NEH_fixed = 14.28 *12 # annual
# avg of summer rates (J,J,A,S) and non-summer rates
EH_rate = (0.10273*4+0.10952*8)/12 # 0.08019, $/kWh, electric rate for electric heating customers
NEH_rate = (0.12168*4+0.12847*8)/12 # 0.09889, $/kWh, electric rate for non-electric heating customers
# marginal carbon emission factor:
elec_CO2_rate = 0.000834702 # metric tons of CO2e/kWh (0.2446 tons/mbtu)

## gas ##
# annual fixed rates = monthly x 12
GH_fixed = 45.32 * 12
NGH_fixed = 21.51 * 12
# variable rates
GH_rate = 0.61648 * NG_rate_multiplier # 0.19477, $/therm, gas rate for NG heating customers
NGH_rate = 0.56758 * NG_rate_multiplier # 0.14964, $/therm, gas rate for non-NG heating customers
# marginal carbon emission factor:
gas_CO2_rate = 0.00532181 # metric tons of CO2e/therm (0.0532 tons/mbtu)

### upgrades that will cause utility rate change: ###########################################
HVAC_upgrades_rate_change = {
    15: [4,5,6], # ASHP
    16: [4,5,6,7], # MSHP
    17: [5,6,7], # ASHP
    18: [5,6,7,8], # MSHP,
    21: [5,6,7], # ASHP
    22: [5,6,7,8], # MSHP,
}

# assign rates accordingly
Elec_heating_types = ['Electricity Baseboard','Electricity ASHP','Electricity Electric Furnace',
                      'Electricity Electric Boiler', 'Electricity Electric Wall Furnace']
NG_heating_types = ['Natural Gas Fuel Wall/Floor Furnace', 'Natural Gas Fuel Furnace',
                    'Natural Gas Fuel Boiler']

res['gas_rate'] = NGH_rate
res['gas_fixed'] = NGH_fixed
res['gas_CO2_rate'] = gas_CO2_rate
res['elec_rate'] = NEH_rate
res['elec_fixed'] = NEH_fixed
res['elec_CO2_rate'] = elec_CO2_rate

res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(NG_heating_types)]).index,
       'gas_rate'] = GH_rate
res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(NG_heating_types)]).index,
       'gas_fixed'] = GH_fixed
res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(Elec_heating_types)]).index,
        'elec_rate'] = EH_rate
res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(Elec_heating_types)]).index,
        'elec_fixed'] = EH_fixed

# assign 0 gas rates to building with no gas use 
res.loc[res['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_fixed'] = 0 
res.loc[res['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_rate'] = 0 
res.loc[res['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_fixed'] = 0 
res.loc[res['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_rate'] = 0

for col in ['gas_rate','gas_fixed','gas_CO2_rate','elec_rate','elec_fixed','elec_CO2_rate']:
    res.loc[res['completed_status']!='Success', col] = np.nan

print(f'Natural gas rate multiplier: {NG_rate_multiplier}')


Natural gas rate multiplier: 1


In [28]:
#### count upgrades
N_upgrades = 15 # <---
export_all_upgrades_to_csv = False # <------

#################################################################################################

MF_dict = {
    '2-4 flats': ['Multi-Family with 2 - 4 Units'],
    'all': ['Multi-Family with 2 - 4 Units', 'Multi-Family with 5+ Units'],
}

print(f'>>> {iter_path} has {N_upgrades:,} packages')
print(f'Natural gas rate multiplier: {NG_rate_multiplier}')
if export_all_upgrades_to_csv:
    print(f'    Exporting upgrades results to {combined_res_csv_path}\n')        
    res.to_csv(os.path.join(combined_res_csv_path, f'results_baseline{fn_ext}.csv'), index=False)
    
## initialize summary tables
summary_upgrades = []

## initialize dB table
all_proto_upgrades = []

package_list = list(range(6, N_upgrades+6)) # <-----
for mf_type, MF in MF_dict.items():
    print(f'\n### Summary for {mf_type} ###################################################')
    
    #### *** subset to MF ***
    res_mf = res[res['build_existing_model.geometry_building_type_recs'].isin(MF)].reset_index(drop=True)
    
    for n in package_list:
        p = pd.read_csv(os.path.join(combined_res_csv_path,
                                         f'package{n:02d}.csv'))
        print(f'\nPackage {n}')
        p['build_existing_model.sample_weight'] = 2173432/40000
        
        ### *** subset to MF ***
        p = p[res['build_existing_model.geometry_building_type_recs'].isin(MF)].reset_index(drop=True)

        ### get sim output at unit level
        p = get_per_unit_sim_output(p, res_mf)

        ### assign utility rates
        p = assign_utility_rates_to_upgrade(n, p, res_mf, HVAC_upgrades_rate_change, for_packages=True)

        ### collapse upgrade cost and lifetime cols
        p = combine_upgrade_cost_and_lifetime(p)
        
        ### add totals metrics:
        p = get_annual_totals(p)

        ### check if upgrade has 0 successful sims
        if len(p[p['completed_status']=='Success']) == 0:
            print(f' * upgrade={n} has 0 successful simulations')

        ### calculate metrics
        p = add_sqft_eui(p, res_mf)
        EUIi = ['gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_kbtupersqft']
        EUIo = ['gas_eui','elec_eui','site_eui']
        for vari, varo in zip(EUIi, EUIo):
            p[f'pct_delta_{varo}'] = ((p[vari]-res[vari])/res[vari]*100)

        # annual energy saving:
        p = get_annual_gas_elec_site_energy_saving(p, res_mf)

        # annual energy cost saving:
        p = get_annual_energy_cost_saving(p, res_mf)
        
        # annual kBtu saved per upgrade cost:
        p['ann_kbtu_saved_per_dollar'] = p['ann_mbtu_site_energy_saving'].divide(
                                p['upgrade_cost'], axis=0)*1000 

        # simple payback
        p['simple_payback'] = p['upgrade_cost']/p['ann_energy_cost_saving']

        # annual metric ton carbon emission savings:
        p = get_annual_metric_ton_co2e_saving(p, res_mf)

        ### check for neg/large paybacks, inf eui, neg carbon savings 
        print_metrics_report(p, has_comparative_payback=False)

        ### subset to only those that have been applied with the upgrades successfully:
        upgrade_name = p['apply_upgrade.upgrade_name'].replace('',np.nan).dropna(axis=0).unique()[0]
        p = p[p['apply_upgrade.applicable']==True].reset_index(drop=True)
        p['package_no'] = n

        ### export upgrade results to csv 
        if export_all_upgrades_to_csv:
            p.to_csv(os.path.join(combined_res_csv_path, f'results_package{n:02d}{fn_ext}.csv'), index=False)
            
        ### add to dB
        if mf_type == 'all':
            all_proto_upgrades.append(p)

        for eui in ['pct_delta_gas_eui','pct_delta_elec_eui','pct_delta_site_eui']:
            p[eui] = p[eui].replace([np.inf, -np.inf], np.nan) # for mean calc

        ### add to summary table
        summ = pd.DataFrame()
        summ.loc[0,'upgrade_no'] = n
        summ.loc[0,'upgrade_name'] = upgrade_name
        summ.loc[0,'mf_type'] = mf_type # <------ *** MF ***
        summ.loc[0,'n_applied'] = len(p)
        summ.loc[0,'n_success'] = len(p[p['completed_status']=='Success'])
        summ.loc[0,'n_fail'] = len(p[p['completed_status']=='Fail'])
        summ['pct_success'] = round(summ['n_success']/summ['n_applied']*100,3)

        p = p[p['completed_status']=='Success'].reset_index(drop=True)
        summ['mean_ann_therm_gas_saving'] = round(p['ann_therm_gas_saving'].mean(), 3)
        summ['mean_ann_kwh_elec_saving'] = round(p['ann_kwh_elec_saving'].mean(), 3)
        summ['mean_ann_mbtu_site_energy_saving'] = round(p['ann_mbtu_site_energy_saving'].mean(), 3)
        summ['mean_pct_delta_gas_eui'] = round(p['pct_delta_gas_eui'].mean(), 3)
        summ['mean_pct_delta_elec_eui'] = round(p['pct_delta_elec_eui'].mean(), 3)
        summ['mean_pct_delta_site_eui'] = round(p['pct_delta_site_eui'].mean(), 3)
        summ['mean_upgrade_cost'] = round(p['upgrade_cost'].mean(),2)
        summ['mean_ann_kbtu_saved_per_dollar'] = round(p['ann_kbtu_saved_per_dollar'].mean(),3) # annual kBtu saved per upgrade cost
        summ['mean_ann_energy_cost_saving'] = round(p['ann_energy_cost_saving'].mean(),2)
        summ['mean_ann_metric_ton_co2e_saving'] = round(p['ann_metric_ton_co2e_saving'].mean(),3)

        summ['median_simple_payback'] = round(p['simple_payback'].median(),3)

        p['simple_payback'] = p['simple_payback'].replace([np.inf, -np.inf], np.nan) # for mean calc
        summ['pct_pos_simple_payback_actual'] = round(len(p[p['simple_payback']>=0])/len(p)*100, 3)
        summ['mean_pos_simple_payback_actual'] = round(p.loc[p['simple_payback']>=0, 'simple_payback'].mean(),3)

        # filter: min 1 cent energy cost savings **
        min_energy_cost_saving = 0.1 # <----- **
        summ['pct_pos_simple_payback_filtered'] = round(len(
            p[(p['simple_payback']>=0) & (p['ann_energy_cost_saving']>=min_energy_cost_saving)]
        )/len(p)*100, 3)
        summ['mean_pos_simple_payback_filtered'] = round(
            p.loc[(p['simple_payback']>=0) & (p['ann_energy_cost_saving']>=min_energy_cost_saving), 
               'simple_payback'].mean(),3)

        summary_upgrades.append(summ)
    
summary_upgrades = pd.concat(summary_upgrades).reset_index(drop=True)
summary_upgrades


>>> cookcnty_packages_02_mf has 15 packages
Natural gas rate multiplier: 1

### Summary for 2-4 flats ###################################################


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (58) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



Package 6
min       -3188.753930
median       14.233351
max       26049.134378
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=6 has 107 simple_payback>100 (including 0 INF)
    *PAYBACK1 - negative*  upgrade=6 has 73 simple_payback<0 (due to negative energy cost saving)
       *EUI - inf* upgrade=6 has 1176 pct_delta_gas_eui=INF (due to fuel introduction from upgrade)
    *CARBON - negative*  upgrade=6 has 75 carbon saving<0 

Package 7
min       -4360.339518
median       23.252151
max       23479.718683
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=7 has 106 simple_payback>100 (including 0 INF)
    *PAYBACK1 - negative*  upgrade=7 has 21 simple_payback<0 (due to negative energy cost saving)
       *EUI - inf* upgrade=7 has 748 pct_delta_gas_eui=INF (due to fuel introduction from upgrade)
    *CARBON - negative*  upgrade=7 has 19 carbon saving<0 

Package 8
min         9.060290
median     31.053131
max       311.820321
Name: simple_pa

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5,8,140) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



Package 15
min      -4.040603e+06
median    2.545969e+01
max       1.818678e+05
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=15 has 392 simple_payback>100 (including 0 INF)
    *PAYBACK1 - negative*  upgrade=15 has 676 simple_payback<0 (due to negative energy cost saving)
       *EUI - inf* upgrade=15 has 748 pct_delta_gas_eui=INF (due to fuel introduction from upgrade)
    *CARBON - negative*  upgrade=15 has 1916 carbon saving<0 

Package 16
min      -1.577160e+04
median    1.942139e+01
max       1.064844e+06
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=16 has 160 simple_payback>100 (including 0 INF)
    *PAYBACK1 - negative*  upgrade=16 has 109 simple_payback<0 (due to negative energy cost saving)
       *EUI - inf* upgrade=16 has 390 pct_delta_gas_eui=INF (due to fuel introduction from upgrade)
    *CARBON - negative*  upgrade=16 has 78 carbon saving<0 


...

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5,8,140) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



Package 15
min      -4.040603e+06
median    2.313102e+01
max       1.818678e+05
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=15 has 830 simple_payback>100 (including 0 INF)
    *PAYBACK1 - negative*  upgrade=15 has 1128 simple_payback<0 (due to negative energy cost saving)
    *CARBON - negative*  upgrade=15 has 3104 carbon saving<0 

Package 16
min      -8.127124e+06
median    2.117380e+01
max       1.064844e+06
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=16 has 283 simple_payback>100 (including 0 INF)
    *PAYBACK1 - negative*  upgrade=16 has 176 simple_payback<0 (due to negative energy cost saving)
    *CARBON - negative*  upgrade=16 has 106 carbon saving<0 


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5,8,140) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



Package 17
min       -32661.315020
median        32.997031
max       106181.155574
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=17 has 703 simple_payback>100 (including 0 INF)
    *PAYBACK1 - negative*  upgrade=17 has 372 simple_payback<0 (due to negative energy cost saving)
    *CARBON - negative*  upgrade=17 has 5222 carbon saving<0 

Package 18
min      -4042.314640
median      30.401061
max       4486.808080
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=18 has 143 simple_payback>100 (including 0 INF)
    *PAYBACK1 - negative*  upgrade=18 has 15 simple_payback<0 (due to negative energy cost saving)
    *CARBON - negative*  upgrade=18 has 412 carbon saving<0 

Package 19
min         4.900637
median     21.347368
max       301.398751
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=19 has 8 simple_payback>100 (including 0 INF)
    *CARBON - negative*  upgrade=19 has 1 carbon saving<0 

Package 20
min        4

,upgrade_no,upgrade_name,mf_type,n_applied,n_success,n_fail,pct_success,mean_ann_therm_gas_saving,mean_ann_kwh_elec_saving,mean_ann_mbtu_site_energy_saving,...,mean_pct_delta_site_eui,mean_upgrade_cost,mean_ann_kbtu_saved_per_dollar,mean_ann_energy_cost_saving,mean_ann_metric_ton_co2e_saving,median_simple_payback,pct_pos_simple_payback_actual,mean_pos_simple_payback_actual,pct_pos_simple_payback_filtered,mean_pos_simple_payback_filtered
0,6.0,Utility Incentive Shell + LEDs,2-4 flats,7712.0,7712.0,0.0,100.000,138.227,731.827,16.473,...,30.709,2448.50,6.320,172.98,1.346,14.233,99.053,25.479,99.053,25.479
1,7.0,Utility Incentive Shell + LEDs - Forced Air,2-4 flats,7712.0,4694.0,0.0,60.866,193.632,627.793,21.501,...,21.083,4339.67,4.957,196.21,1.554,23.252,99.553,36.366,99.553,36.366
2,8.0,Utility Incentive Shell + LEDs + Appliances - ...,2-4 flats,7712.0,4694.0,0.0,60.866,296.218,890.700,32.654,...,6.026,8561.53,3.649,291.94,2.320,31.053,100.000,35.502,100.000,35.502
3,9.0,Utility Incentive Shell + WH - Hydronic,2-4 flats,7712.0,750.0,0.0,9.725,417.845,518.846,43.545,...,5.231,10910.82,3.871,323.07,2.657,35.083,100.000,38.394,100.000,38.394
4,10.0,Utility Incentive Shell + LEDs + Appliances - ...,2-4 flats,7712.0,750.0,0.0,9.725,420.511,730.521,44.534,...,3.837,12924.21,3.358,351.43,2.848,38.315,100.000,40.898,100.000,40.898
5,11.0,Comprehensive - Forced Air + WH + Shell,2-4 flats,7712.0,4694.0,0.0,60.866,341.401,760.883,36.728,...,1.016,12559.44,2.886,302.65,2.452,43.586,100.000,50.657,100.000,50.657
6,12.0,Comprehensive - Forced Air + WH + Insulation +...,2-4 flats,7712.0,4694.0,0.0,60.866,304.507,677.294,32.754,...,5.907,7775.63,4.121,269.99,2.186,30.328,100.000,37.596,100.000,37.596
7,13.0,Comprehensive - Hydronic + WH + Shell,2-4 flats,7712.0,750.0,0.0,9.725,466.167,532.803,48.424,...,-1.128,16713.15,2.870,354.63,2.926,48.359,100.000,52.884,100.000,52.884
8,14.0,Comprehensive - Hydronic + WH + Shell + Applia...,2-4 flats,7712.0,750.0,0.0,9.725,429.455,495.581,44.626,...,3.774,12170.64,3.601,327.30,2.699,38.222,100.000,42.942,100.000,42.942
9,15.0,Comprehensive - ASHP,2-4 flats,7712.0,4694.0,0.0,60.866,796.099,-5001.934,62.524,...,-31.241,9981.74,6.078,257.08,0.062,25.460,85.599,135.079,85.577,89.838


In [29]:
### export UPGRADE summary
summary_upgrades.to_csv(os.path.join(result_path,'processed results',f'upgrades_summary{fn_ext}.csv'), index=False)
print(f'UPGRADE summary table saved to:\n  {os.path.join(result_path, "processed results")}')

UPGRADE summary table saved to:
  /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_mf/processed results


In [30]:
### optional - export specific upgrade parquet as csv
save_to_csv = False # <-----
upgrade_list = range(1, 28+1) # [8] # <----- list of upgrades to convert to csv

if save_to_csv:
    for n in upgrade_list:
        p = pd.read_parquet(os.path.join(result_path,'upgrades',
                                         f'upgrade={n}/results_up{n:02d}.parquet'))
        p.to_csv(os.path.join(result_path,'upgrades',
                                         f'upgrade={n}/results_up{n:02d}.csv'), index=False)
        print(f'upgrade {n:02d} parquet file exported as csv')


### *Combine results by building prototypes from Elevate Energy into database

In [31]:
# (1) combine baseline results by non-split-level prototypes (N=12)
print(f'Natural gas rate multiplier: {NG_rate_multiplier}')
### add totals metrics:
res = get_annual_totals(res)

res_proto_upgrades = res.copy()

res_proto_upgrades['package_no'] = 0
res_proto_upgrades['apply_upgrade.upgrade_name'] = 'Baseline'

# wt_2_4_flats = 26.4/(26.4+16+8.9)*100
# res_proto_upgrades['build_existing_model.sample_weight'] = np.where(
#     res_proto_upgrades['build_existing_model.geometry_building_type_recs']=='Multi-Family with 2 - 4 Units',
#     wt_2_4_flats,
#     100-wt_2_4_flats
# )

display(res_proto_upgrades)

Natural gas rate multiplier: 1


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,elec_rate,elec_fixed,elec_CO2_rate,ann_gas_cost,ann_elec_cost,ann_energy_cost,ann_metric_ton_co2e_gas,ann_metric_ton_co2e_elec,ann_metric_ton_co2e,package_no
0,1,188,2021-02-14 08:59:53,2021-02-14 09:01:21,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,1499.216472,1542.883617,3042.100089,8.247359,9.070943,17.318303,0
1,2,123,2021-02-14 15:22:23,2021-02-14 15:37:42,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,861.385127,884.825411,1746.210538,2.741232,4.718697,7.459929,0
2,3,175,2021-02-15 01:22:58,2021-02-15 01:26:24,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,1312.566014,1593.959847,2906.525860,6.636085,9.408750,16.044835,0
3,4,119,2021-02-14 09:19:39,2021-02-14 09:21:10,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,949.394593,1010.551932,1959.946526,3.500981,5.550223,9.051204,0
4,5,63,2021-02-14 10:36:00,2021-02-14 10:55:40,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.107257,188.40,0.000835,0.000000,1741.910478,1741.910478,0.000000,12.089862,12.089862,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,35,2021-02-14 08:02:24,2021-02-14 08:09:40,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.107257,188.40,0.000835,270.200589,2809.248646,3079.449236,0.113271,20.396192,20.509464,0
20755,20756,219,2021-02-14 12:10:30,2021-02-14 12:54:43,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.107257,188.40,0.000835,269.051469,1543.412363,1812.463832,0.102497,10.545093,10.647590,0
20756,20757,223,2021-02-14 14:28:38,2021-02-14 14:43:34,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,841.749774,931.651564,1773.401338,2.571729,5.028394,7.600123,0
20757,20758,243,2021-02-15 00:30:42,2021-02-15 00:41:24,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,0.000000,2283.073704,2283.073704,0.000000,13.966391,13.966391,0


In [32]:
# (2) combine upgrade results by non-split-level prototypes (N=12)
print(f'Natural gas rate multiplier: {NG_rate_multiplier}')

all_proto_upgrades = pd.concat(all_proto_upgrades, axis=0).reset_index(drop=True)
all_proto_upgrades


Natural gas rate multiplier: 1


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,ann_gas_cost_saving,ann_elec_cost_saving,ann_energy_cost_saving,ann_kbtu_saved_per_dollar,simple_payback,ann_metric_ton_co2e_saving_gas,ann_metric_ton_co2e_saving_elec,ann_metric_ton_co2e_saving,package_no,apply_upgrade.upgrade_part
0,1,136,2021-02-14 11:16:43,2021-02-14 11:19:28,Success,True,Utility Incentive Shell + LEDs,NaN,True,20.000000,...,128.051788,134.860067,262.911855,8.347154,11.124108,1.105417,0.891934,1.997350,6,NaN
1,2,118,2021-02-14 14:31:38,2021-02-14 14:48:43,Success,True,Utility Incentive Shell + LEDs,NaN,True,12.000000,...,29.831736,115.636438,145.468174,4.374040,12.516827,0.257525,0.764793,1.022318,6,NaN
2,3,59,2021-02-14 20:46:03,2021-02-14 20:50:45,Success,True,Utility Incentive Shell + LEDs,NaN,True,20.000000,...,62.529416,202.148794,264.678210,4.611666,12.785337,0.539790,1.336966,1.876756,6,NaN
3,4,13,2021-02-14 17:43:08,2021-02-14 17:45:51,Success,True,Utility Incentive Shell + LEDs,NaN,True,20.000000,...,24.045922,17.513186,41.559108,3.204190,32.839973,0.207578,0.115828,0.323406,6,NaN
4,5,157,2021-02-14 12:10:40,2021-02-14 12:32:37,Success,True,Utility Incentive Shell + LEDs,NaN,True,10.000000,...,0.000000,207.747622,207.747622,4.328490,7.349639,0.000000,1.616751,1.616751,6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311376,20755,128,2021-02-14 18:05:41,2021-02-14 18:21:12,Success,True,Electrification - Electric to MSHP,NaN,True,20.000000,...,270.200589,1660.143889,1930.344479,2.396076,11.878667,0.113271,12.919714,13.032985,20,Electrification - Electric to MSHP - Part 3
311377,20756,181,2021-02-14 17:15:39,2021-02-14 18:20:07,Success,True,Electrification - Electric to MSHP,NaN,True,6.666667,...,269.051469,528.224281,797.275750,1.231677,19.073415,0.102497,4.110792,4.213289,20,Electrification - Electric to MSHP - Part 6
311378,20757,214,2021-02-14 23:47:46,2021-02-14 23:48:34,Invalid,True,Electrification - Electric to MSHP,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,Electrification - Electric to MSHP - Part 6
311379,20758,131,2021-02-14 18:17:06,2021-02-14 18:40:32,Success,True,Electrification - Electric to MSHP,NaN,True,12.000000,...,0.000000,1416.515011,1416.515011,2.640356,10.239573,0.000000,9.368506,9.368506,20,Electrification - Electric to MSHP - Part 3


In [33]:
# (3) combine upgrade and baseline (results by non-split-level prototypes (N=12))

cols = set(res_proto_upgrades.columns).intersection(set(all_proto_upgrades.columns))
all_proto_upgrades = pd.concat([all_proto_upgrades, res_proto_upgrades[cols]], axis=0).reset_index(drop=True)

# (4) save
print('"all_proto_upgrades" df saved')
all_proto_upgrades.to_csv(
    os.path.join(result_path,'processed results',f'upgrades_dB{fn_ext}.csv'), index=False)

display(all_proto_upgrades)


"all_proto_upgrades" df saved


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,ann_gas_cost_saving,ann_elec_cost_saving,ann_energy_cost_saving,ann_kbtu_saved_per_dollar,simple_payback,ann_metric_ton_co2e_saving_gas,ann_metric_ton_co2e_saving_elec,ann_metric_ton_co2e_saving,package_no,apply_upgrade.upgrade_part
0,1,136,2021-02-14 11:16:43,2021-02-14 11:19:28,Success,True,Utility Incentive Shell + LEDs,NaN,True,20.000000,...,128.051788,134.860067,262.911855,8.347154,11.124108,1.105417,0.891934,1.997350,6,NaN
1,2,118,2021-02-14 14:31:38,2021-02-14 14:48:43,Success,True,Utility Incentive Shell + LEDs,NaN,True,12.000000,...,29.831736,115.636438,145.468174,4.374040,12.516827,0.257525,0.764793,1.022318,6,NaN
2,3,59,2021-02-14 20:46:03,2021-02-14 20:50:45,Success,True,Utility Incentive Shell + LEDs,NaN,True,20.000000,...,62.529416,202.148794,264.678210,4.611666,12.785337,0.539790,1.336966,1.876756,6,NaN
3,4,13,2021-02-14 17:43:08,2021-02-14 17:45:51,Success,True,Utility Incentive Shell + LEDs,NaN,True,20.000000,...,24.045922,17.513186,41.559108,3.204190,32.839973,0.207578,0.115828,0.323406,6,NaN
4,5,157,2021-02-14 12:10:40,2021-02-14 12:32:37,Success,True,Utility Incentive Shell + LEDs,NaN,True,10.000000,...,0.000000,207.747622,207.747622,4.328490,7.349639,0.000000,1.616751,1.616751,6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332135,20755,35,2021-02-14 08:02:24,2021-02-14 08:09:40,Success,None,Baseline,,True,20.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
332136,20756,219,2021-02-14 12:10:30,2021-02-14 12:54:43,Success,None,Baseline,,True,6.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
332137,20757,223,2021-02-14 14:28:38,2021-02-14 14:43:34,Success,None,Baseline,,True,4.285714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
332138,20758,243,2021-02-15 00:30:42,2021-02-15 00:41:24,Success,None,Baseline,,True,12.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [34]:
## save metadata for dB
if NG_rate_multiplier==1:
    cols2 = sorted((set(res_proto_upgrades.columns)-set(cols)).union(
                   set(['building_id','completed_status','package_no']))
                  )
    res_proto_meta = res_proto_upgrades[cols2]
    res_proto_meta = res_proto_meta[res_proto_meta['completed_status']=='Success'].reset_index(drop=True)
    
    cols_to_drop = [x for x in res_proto_meta.columns if 
                    x.startswith('simulation_output_report.option') or
                    x == 'simulation_output_report.upgrade_cost_usd']
    
    res_proto_meta = res_proto_meta.drop(cols_to_drop, axis=1)

    res_proto_meta.to_csv(
        os.path.join(result_path,'processed results',f'upgrades_dB_meta{fn_ext}.csv'), index=False)

    print('"res_proto_meta" df saved')
    display(res_proto_meta)
    

"res_proto_meta" df saved


,build_existing_model.ahs_region,build_existing_model.applicable,build_existing_model.ashrae_iecc_climate_zone_2004,build_existing_model.bathroom_spot_vent_hour,build_existing_model.bedrooms,build_existing_model.building_america_climate_zone,build_existing_model.ceiling_fan,build_existing_model.census_division,build_existing_model.census_region,build_existing_model.chicago,...,build_existing_model.vintage,build_existing_model.vintage_acs,build_existing_model.water_heater,build_existing_model.window_areas,build_existing_model.windows,building_id,completed_status,package_no,stories,vintage_ee
0,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour14,3,Cold,Standard Efficiency,East North Central,Midwest,City of Chicago,...,1940s,1940-59,Gas Standard,F12 B12 L12 R12,2+ Pane,1,Success,0,1: <2 stories,2: 1942-1978
1,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour7,2,Cold,Standard Efficiency,East North Central,Midwest,City of Chicago,...,1950s,1940-59,Gas Standard,F30 B30 L30 R30,2+ Pane,2,Success,0,2: 2+ stories,2: 1942-1978
2,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour22,5,Cold,None,East North Central,Midwest,City of Chicago,...,<1940,<1940,Gas Standard,F9 B9 L9 R9,1 Pane,3,Success,0,1: <2 stories,1: pre-1942
3,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour7,2,Cold,Standard Efficiency,East North Central,Midwest,City of Chicago,...,1950s,1940-59,Gas Standard,F9 B9 L9 R9,1 Pane,4,Success,0,1: <2 stories,2: 1942-1978
4,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour1,2,Cold,Standard Efficiency,East North Central,Midwest,City of Chicago,...,1990s,1980-99,Electric Standard,F15 B15 L15 R15,2+ Pane,5,Success,0,2: 2+ stories,3: post-1978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour19,2,Cold,Standard Efficiency,East North Central,Midwest,Non-City Cook County,...,1970s,1960-79,Electric Standard,F18 B18 L18 R18,2+ Pane,20755,Success,0,2: 2+ stories,2: 1942-1978
20755,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour19,1,Cold,Standard Efficiency,East North Central,Midwest,City of Chicago,...,1990s,1980-99,Electric Standard,F30 B30 L30 R30,2+ Pane,20756,Success,0,2: 2+ stories,3: post-1978
20756,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour7,3,Cold,Standard Efficiency,East North Central,Midwest,City of Chicago,...,2000s,2000-09,Gas Standard,F18 B18 L18 R18,2+ Pane,20757,Success,0,2: 2+ stories,3: post-1978
20757,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour22,2,Cold,Standard Efficiency,East North Central,Midwest,Non-City Cook County,...,1980s,1980-99,Electric Standard,F15 B15 L15 R15,2+ Pane,20758,Success,0,2: 2+ stories,3: post-1978


In [35]:
### check

In [36]:
# SFD
iteration_sfd = '02' # <----- options: 02, 02_2018, 02_2019
iter_path_sfd = f'cookcnty_packages_{iteration_sfd}'
result_dir_sfd = '/Users/lliu2/Documents/Chicago retrofits/ResStock results'
result_path_sfd = os.path.join(result_dir_sfd, iter_path_sfd)

all_proto_upgrades_sfd = pd.read_csv(
    os.path.join(result_path_sfd,'processed results',f'upgrades_dB{fn_ext}.csv')
)
res_proto_meta_sfd = pd.read_csv(
        os.path.join(result_path_sfd,'processed results',f'upgrades_dB_meta{fn_ext}.csv')
)
all_proto_upgrades_sfd = all_proto_upgrades_sfd.join(res_proto_meta_sfd.drop(
    columns=['completed_status', 'package_no', 'HousingGroupNo', 'HousingGroupName'])\
                        .set_index('building_id'), on='building_id')

print(len(all_proto_upgrades_sfd))

# MF
all_proto_upgrades = pd.read_csv(
    os.path.join(result_path,'processed results',f'upgrades_dB{fn_ext}.csv')
)
res_proto_meta = pd.read_csv(
        os.path.join(result_path,'processed results',f'upgrades_dB_meta{fn_ext}.csv')
)
all_proto_upgrades = all_proto_upgrades.join(res_proto_meta.drop(
    columns=['completed_status', 'package_no'])\
                        .set_index('building_id'), on='building_id')

all_proto_upgrades['building_id'] += 16777
print(len(all_proto_upgrades))

# combine
all_proto_upgrades = pd.concat([all_proto_upgrades_sfd, all_proto_upgrades], axis=0)

# city only
all_proto_upgrades = all_proto_upgrades.loc[
    (all_proto_upgrades['build_existing_model.chicago']=='City of Chicago') &
    (all_proto_upgrades['simulation_output_report.applicable']==True)
].reset_index(drop=True)

# add kWh and Therm conversion to mmBtu
all_proto_upgrades['ann_mbtu_elec_saving'] = all_proto_upgrades['ann_kwh_elec_saving']*0.003412142
all_proto_upgrades['ann_mbtu_gas_saving'] = all_proto_upgrades['ann_therm_gas_saving']*0.1

all_proto_upgrades

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5,141) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


142755


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5,58,139) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


332140


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,build_existing_model.vacancy_status,build_existing_model.vintage,build_existing_model.vintage_acs,build_existing_model.water_heater,build_existing_model.window_areas,build_existing_model.windows,stories,vintage_ee,ann_mbtu_elec_saving,ann_mbtu_gas_saving
0,259,59,2021-02-05 04:24:33,2021-02-05 04:27:38,Success,True,Utility Incentive Shell + LEDs,NaN,True,20.000000,...,Occupied,1990s,1980-99,Gas Standard,F12 B12 L12 R12,2+ Pane,1: <2 stories,3: post-1978,3.695921,7.090582
1,333,230,2021-02-05 07:11:17,2021-02-05 07:13:33,Success,True,Utility Incentive Shell + LEDs,NaN,True,20.000000,...,Occupied,1990s,1980-99,Gas Standard,F12 B12 L12 R12,2+ Pane,1: <2 stories,3: post-1978,0.984517,22.120858
2,357,211,2021-02-05 06:23:03,2021-02-05 06:26:07,Success,True,Utility Incentive Shell + LEDs,NaN,True,20.000000,...,Occupied,1990s,1980-99,Gas Standard,F9 B9 L9 R9,2+ Pane,1: <2 stories,3: post-1978,1.421233,17.864757
3,367,116,2021-02-05 00:46:48,2021-02-05 00:49:59,Success,True,Utility Incentive Shell + LEDs,NaN,True,20.000000,...,Occupied,1980s,1980-99,Gas Standard,F9 B9 L9 R9,2+ Pane,1: <2 stories,3: post-1978,6.786045,16.676483
4,614,62,2021-02-05 03:29:48,2021-02-05 03:32:28,Success,True,Utility Incentive Shell + LEDs,NaN,True,20.000000,...,Occupied,2000s,2000-09,Gas Standard,F18 B18 L18 R18,2+ Pane,1: <2 stories,3: post-1978,2.817398,1.705625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143206,37529,96,2021-02-14 13:17:01,2021-02-14 13:32:04,Success,NaN,Baseline,NaN,True,20.000000,...,Occupied,<1940,<1940,Gas Standard,F12 B12 L12 R12,1 Pane,2: 2+ stories,1: pre-1942,NaN,NaN
143207,37530,102,2021-02-14 10:57:34,2021-02-14 11:10:23,Success,NaN,Baseline,NaN,True,20.000000,...,Occupied,<1940,<1940,Gas Standard,F30 B30 L30 R30,1 Pane,2: 2+ stories,1: pre-1942,NaN,NaN
143208,37533,219,2021-02-14 12:10:30,2021-02-14 12:54:43,Success,NaN,Baseline,NaN,True,6.666667,...,Occupied,1990s,1980-99,Electric Standard,F30 B30 L30 R30,2+ Pane,2: 2+ stories,3: post-1978,NaN,NaN
143209,37534,223,2021-02-14 14:28:38,2021-02-14 14:43:34,Success,NaN,Baseline,NaN,True,4.285714,...,Occupied,2000s,2000-09,Gas Standard,F18 B18 L18 R18,2+ Pane,2: 2+ stories,3: post-1978,NaN,NaN


In [37]:
all_proto_upgrades.groupby(['package_no','simulation_output_report.applicable']).agg({
    'building_id':'count',
    'build_existing_model.sample_weight':'sum',
})

,,building_id,build_existing_model.sample_weight
package_no,simulation_output_report.applicable,,
0,True,21207,1.152299e+06
6,True,19778,1.074653e+06
7,True,12707,6.904450e+05
8,True,12707,6.904450e+05
9,True,1858,1.009559e+05
10,True,1858,1.009559e+05
11,True,12707,6.904450e+05
12,True,12707,6.904450e+05
13,True,1858,1.009559e+05


In [38]:
all_proto_upgrades.loc[
       all_proto_upgrades['package_no'].isin(packages_to_combine),:]

NameError: name 'packages_to_combine' is not defined

In [ ]:
scenarios = {
    'Scenario 1: Basic Utility Improvements': [6],
    'Scenario 2: Comprehensive Utility Improvements': [8,10],
    'Scenario 3: Deep Efficiency, HVAC Electrification': [15, 16],
    'Scenario 4: Full Electrification': [17, 18, 19, 20]
    
}

scenario_result_by_fuel, scenario_result = [], []
for scenario_name, packages_to_combine in scenarios.items():
    result_by_fuel = all_proto_upgrades.loc[
       all_proto_upgrades['package_no'].isin(packages_to_combine),[
            'building_id',
           ## by fuel type
            'ann_mbtu_elec_saving', 
            'ann_mbtu_gas_saving', 
            'ann_elec_cost_saving', 
            'ann_gas_cost_saving', 
            'ann_metric_ton_co2e_saving_elec', 
            'ann_metric_ton_co2e_saving_gas', 
        ]
    ].groupby('building_id').max().reset_index()
    
    result = all_proto_upgrades.loc[
       all_proto_upgrades['package_no'].isin(packages_to_combine),[
            'building_id',
           # total
            'ann_mbtu_site_energy_saving',
            'ann_energy_cost_saving',
            'ann_metric_ton_co2e_saving',
        ]
    ].groupby('building_id').max().reset_index()
    
    result_by_fuel = result_by_fuel.agg({
        # by fuel type
        'ann_mbtu_elec_saving':'sum', 
        'ann_mbtu_gas_saving':'sum', 
        'ann_elec_cost_saving':'sum', 
        'ann_gas_cost_saving':'sum',
        'ann_metric_ton_co2e_saving_elec':'sum',
        'ann_metric_ton_co2e_saving_gas':'sum',
    })*wt
    
    result = result.agg({   
        # total
        'ann_mbtu_site_energy_saving':'sum',
        'ann_energy_cost_saving':'sum',
        'ann_metric_ton_co2e_saving':'sum',
    })*wt
    
    scenario_result_by_fuel.append(
        result_by_fuel.rename('\n'.join(wrap(scenario_name, 30)))
    )
    
    scenario_result.append(
        result.rename('\n'.join(wrap(scenario_name, 30)))
    )

scenario_result = pd.concat(scenario_result, axis=1).transpose()
scenario_result = scenario_result.rename(columns={
    'ann_energy_cost_saving': 'Utility Bill Savings',
    'ann_mbtu_site_energy_saving': 'Energy Savings (mmBtu)',
    'ann_metric_ton_co2e_saving': 'CO2 Savings (Metric Ton)',
})

scenario_result_by_fuel = pd.concat(scenario_result_by_fuel, axis=1).reset_index()
display(scenario_result_by_fuel)
scenario_result_by_fuel['fuel type'] = 'Electricity'
scenario_result_by_fuel.loc[scenario_result_by_fuel['index'].str.contains('gas'), 'fuel type'] = 'Natural Gas'

scenario_result_by_fuel['index'] = scenario_result_by_fuel['index'].apply(lambda x: ('').join(
    x for x in 
    ('').join(x for x in x.split('_elec')).split('_gas')
)
                              )

scenario_result_by_fuel = scenario_result_by_fuel.set_index(['index', 'fuel type']).unstack().transpose()
scenario_result_by_fuel = scenario_result_by_fuel.rename(columns={
    'ann_cost_saving': 'Utility Bill Savings',
    'ann_mbtu_saving': 'Energy Savings (mmBtu)',
    'ann_metric_ton_co2e_saving': 'CO2 Savings (Metric Ton)',
})

display(scenario_result)
scenario_result_by_fuel

In [ ]:
# electricity emission is reduced by 30% by 2030
scenario_result_by_fuel_2030 = scenario_result_by_fuel.reset_index()
display(scenario_result_by_fuel_2030)

scenario_result_by_fuel_2030.loc[
    scenario_result_by_fuel_2030['fuel type']=='Electricity', 'CO2 Savings (Metric Ton)'
] *= 0.7

scenario_result_by_fuel_2030

In [ ]:
scenario_result_2030 = scenario_result_by_fuel_2030.groupby(['level_0']).sum()
scenario_result_2030.index.name = None
scenario_result_2030

In [ ]:
import matplotlib.pyplot as plt
from textwrap import wrap

cols = ['Energy Savings (mmBtu)', 'Utility Bill Savings', 'CO2 Savings (Metric Ton)']

fig, axs = plt.subplots(len(cols), sharex=True, figsize=(11,9))
for i in range(len(cols)):
    col = cols[i]
    scenario_result_by_fuel[col].unstack(level=1).plot(kind='bar', stacked=True, ax=axs[i], legend=False, rot=0)
    scenario_result[col].rename('Net').plot(kind='bar', ax=axs[i], legend=False, rot=0, fill=False,
                                              edgecolor='Tab:green', lw=3)
    axs[i].set_ylabel('\n'.join(wrap(col, 30)))
    axs[i].axhline(y=0, c='gray', lw=1, label='_nolegend_')

handles, labels = axs[i].get_legend_handles_labels()
fig.legend(handles,     # The line objects
           labels=labels,   # The labels for each line
           loc="lower center",   # Position of legend
           borderaxespad=None,    # Small spacing around legend box
           title="heating_fuel",  # Title for the legend
           ncol=3,
           bbox_to_anchor=(0.5, 1)
           )

plt.tight_layout()
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0)



In [ ]:
cols = ['Energy Savings (mmBtu)', 'Utility Bill Savings', 'CO2 Savings (Metric Ton)']

fig, axs = plt.subplots(len(cols), sharex=True, figsize=(11,9))

ylims = []
for i in range(len(cols)):
    col = cols[i]
    scenario_result_by_fuel[col].unstack(level=1).plot(kind='bar', stacked=True, ax=axs[i], legend=False, rot=0)
    
    ylims.append(axs[i].get_ylim())
    axs[i].set_ylabel('\n'.join(wrap(col, 30)))
    axs[i].axhline(y=0, c='gray', lw=1, label='_nolegend_')
    
handles, labels = axs[i].get_legend_handles_labels()
fig.legend(handles,     # The line objects
           labels=labels,   # The labels for each line
           loc="lower center",   # Position of legend
           borderaxespad=None,    # Small spacing around legend box
           title="heating_fuel",  # Title for the legend
           ncol=2,
           bbox_to_anchor=(0.5, 1)
           )

plt.tight_layout()
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0)


In [ ]:
ymaxs

In [ ]:
cols = ['Energy Savings (mmBtu)', 'Utility Bill Savings', 'CO2 Savings (Metric Ton)']

fig, axs = plt.subplots(len(cols), sharex=True, figsize=(11,9))
for i in range(len(cols)):
    col = cols[i]
    scenario_result[col].rename('Net').plot(kind='bar', ax=axs[i], legend=False, rot=0,
                                              color='Tab:green')
    axs[i].set_ylabel('\n'.join(wrap(col, 30)))
    axs[i].set_ylim(bottom=ylims[i][0], top=ylims[i][1])
    axs[i].axhline(y=0, c='gray', lw=1, label='_nolegend_')
    
handles, labels = axs[i].get_legend_handles_labels()
fig.legend(handles,     # The line objects
           labels=labels,   # The labels for each line
           loc="lower center",   # Position of legend
           borderaxespad=None,    # Small spacing around legend box
           title="heating_fuel",  # Title for the legend
           ncol=1,
           bbox_to_anchor=(0.5, 1)
           )
plt.tight_layout()
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0)


In [ ]:
### by 2030
cols = ['Energy Savings (mmBtu)', 'Utility Bill Savings', 'CO2 Savings (Metric Ton)']

fig, axs = plt.subplots(len(cols), sharex=True, figsize=(11,9))

for i in range(len(cols)):
    col = cols[i]
    scenario_result_2030[col].rename('Net').plot(kind='bar', ax=axs[i], legend=False, rot=0,
                                              color='Tab:green')
    axs[i].set_ylabel('\n'.join(wrap(col, 30)))
    axs[i].set_ylim(bottom=ylims[i][0], top=ylims[i][1])
    axs[i].axhline(y=0, c='gray', lw=1, label='_nolegend_')
    
handles, labels = axs[i].get_legend_handles_labels()
fig.legend(handles,     # The line objects
           labels=labels,   # The labels for each line
           loc="lower center",   # Position of legend
           borderaxespad=None,    # Small spacing around legend box
           title="heating_fuel",  # Title for the legend
           ncol=1,
           bbox_to_anchor=(0.5, 1)
           )
plt.tight_layout()
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0)


In [ ]:
all_proto_upgrades.columns[-30:]

In [ ]:
wt = 2173432 /40000
print('ann_mbtu_site_energy_saving: ')
print(
    (all_proto_upgrades.loc[all_proto_upgrades['package_no']==6,
                            'ann_mbtu_site_energy_saving'].sum())*wt
)

print('ann_metric_ton_co2e_saving: ')
print(
    (all_proto_upgrades.loc[all_proto_upgrades['package_no']==6,
                            'ann_metric_ton_co2e_saving'].sum())*wt
)

### 2.3. Visualize UPGRADE prototype summary 
#### 2.3.1. plot mean metrics values
Run cell below to load df for plots

In [ ]:
load_df_for_plot_below_from_file = False # <-----
    
if load_df_for_plot_below_from_file:
    
    NG_rate_multiplier = 1 # 1 or 3 <----- This controls what to plot in '2.3.5.'
 
    if NG_rate_multiplier > 1:
        fn_ext = f'_{NG_rate_multiplier}x_gas_prices' # file name extension to add to relevant results
    else:
        fn_ext = ''
    
    filename = os.path.join(result_path,'processed results', f'upgrades_summary{fn_ext}.csv')
    if not os.path.exists(filename):
        print(f'"summary_upgrades{fn_ext}" df does not exist, check if Multi-Family buildings have been modeled.')
    else:
        summary_upgrades = pd.read_csv(filename)
    
        print(f'"summary_upgrades{fn_ext}" df loaded from file.')
        N_upgrades = summary_upgrades['upgrade_no'].nunique()
        print(f'   total number of upgrades: {N_upgrades}')

        display(summary_upgrades)
    

In [ ]:
### plot MF-all & MF-2-4 flats (Cook County Avg) ###

for mf_type in summary_upgrades['mf_type'].unique():
    summ_ups = summary_upgrades[
        (~summary_upgrades['upgrade_no'].isnull()) & (summary_upgrades['mf_type']==mf_type)
         ].reset_index(drop=True)

    N_upgrades=15
    upgrade_sets = {
        'Packages': range(6, N_upgrades+6),
    }
    upgrade_set_colors = {
        'Packages': 'tab:blue',
    }
    # create subfolder for plots:

    plot_path5 = os.path.join(plot_path, f'mean_values_mf_{mf_type}_cook_cnty_avg')
    if not os.path.exists(plot_path5):
        os.mkdir(plot_path5)
    print(f'plot dir: {plot_path5}\n')

    if NG_rate_multiplier == 1: 
        metrics_to_plot = list(x for x in summary_upgrades.columns if 
                               x.startswith('mean') or x.startswith('median'))
    else:
        metrics_to_plot = ['mean_ann_energy_cost_saving', 
                           'median_simple_payback',
                           'mean_pos_simple_payback_filtered']

    for metric in metrics_to_plot:
        title_ext = '' if fn_ext == '' else f' ({fn_ext.replace("_"," ").lstrip()})'
        title = f'Multi-Family {mf_type} per Unit: {metric}{title_ext}'
        print(f'Plotting {title}...')

        for n, up in enumerate(upgrade_sets.keys(),1):
            print(f'    - Group{n}')

            groupi = summ_ups.loc[:, ['upgrade_name', metric]
                                 ].set_index(['upgrade_name']).replace([np.inf, -np.inf], np.nan)

            if metric == 'median_comparative_payback':
                groupi = groupi.replace(0,np.nan).sort_values(by = metric) # make 0 nan
            else:
                groupi = groupi.sort_values(by = metric)
            groupi.index.name = up

            ht = len(groupi)
            fig, ax = plt.subplots(figsize=(6, 0.3*ht))
            groupi.plot.barh(legend=False, title=title, ax=ax)

            # For each bar: Place a label ############################
            rects = ax.patches
            for rect in rects:
                # Get X and Y placement of label from rect.
                x_value = rect.get_width()
                y_value = rect.get_y() + rect.get_height() / 2

                # Number of points between bar and label. Change to your liking.
                space = 5
                # Vertical alignment for positive values
                ha = 'left'

                # If value of bar is negative: Place label left of bar
                if x_value < 0:
                    # Invert space to place label to the left
                    space *= -1
                    # Horizontally align label at right
                    ha = 'right'

                # Use X value as label and format number with one decimal place
                label = "{:.1f}".format(x_value)

                # Create annotation
                plt.annotate(
                    label,                      # Use `label` as label
                    (x_value, y_value),         # Place label at end of the bar
                    xytext=(space, 0),          # Horizontally shift label by `space`
                    textcoords="offset points", # Interpret `xytext` as offset in points
                    va='center',                # Vertically center label
                    ha=ha,                      # Horizontally align label differently for
                    fontsize=8)                 # positive and negative values. 

            ax.margins(x=0.2)    
            ax.axvline(x=0, linestyle='-', color='gray')
            if (metric[-3:]=='eui') & (ax.get_xlim()[0] < -50):
                ax.axvline(x=-50, linestyle='--', color='darkred')

            filename = f'MF-{mf_type}_upgrades_{iteration}_{metric[5:]}_group{n}{fn_ext}.pdf'
            fig.savefig(os.path.join(plot_path5, filename),
                        bbox_inches='tight')
        